# Intro

Import of used libraries   

In [147]:
import re
import unidecode
import unicodedata
import pandas as pd
import numpy as np
from typing import List

Importing dictionary from districts_subdistricts_dict.py for normalization of districts

In [148]:

from districts_subdistricts_dict import districts_subdistricts_dict

Look on dataframes from property platforms (scraped, example of morizon scraper in repo):

    - morizon
    - otodom
    - no (Nieruchomosci Online)

## Class collecting dataframes from portal databases, storing methods used for all sources

In [149]:
class PortalData:

    def __init__(self, platform_name: str) -> None:

        self.platform_name = platform_name
        self.file_path = f"data/{platform_name}/{platform_name}.csv"
        self.cleaned_file_path = f"data/archive/{platform_name}/{platform_name}.csv"
        self.data_frame = pd.DataFrame()

    @property
    def df(self):
        return self.data_frame
    
    @df.setter
    def df(self, value):
        self._df = value
        
    def load_data(self) -> pd.DataFrame:
        self.data_frame = pd.read_csv(self.file_path)
        return self.data_frame

    def show_dataframe(self) -> pd.DataFrame:
        return self.data_frame.sample(10)

    def clean_columns(self) -> set:
        return set(self.data_frame.columns.str.strip().str.lower())

    def clean_text_data(self, columns: List[str]) -> pd.DataFrame:
        """
        Czyści dane tekstowe w określonych kolumnach DataFrame, zamieniając polskie znaki na ich uniwersalne odpowiedniki
        oraz zachowując spacje między słowami. Konwertuje tekst na małe litery. Pomija wartości numeryczne.

        Args:
        - columns (List[str]): Lista kolumn do oczyszczenia.

        Returns:
        - pd.DataFrame: DataFrame z oczyszczonymi kolumnami.
        """
        
        def clean_text(text):
            if isinstance(text, str):
                text = unicodedata.normalize('NFKD', text)
                text = ''.join([c for c in text if not unicodedata.combining(c)])
                text = text.lower()
                text = ''.join(char if char.isalnum() or char.isspace() else ' ' for char in text)
                return ' '.join(text.split())
            else:
                return text
        
        for column in columns:
            if column in self.data_frame.columns and self.data_frame[column].dtype == object:  # Assumes 'object' dtype for text data
                self.data_frame[column] = self.data_frame[column].apply(clean_text)
        return self.data_frame
    
    def create_nan_column(self, column_names: List[str]):
        for column_name in column_names:
            new_column_name = f"{column_name}_nan"
            self.data_frame[new_column_name] = self.data_frame[column_name].apply(lambda x: 1 if x == 'none' else 0)
        return self.data_frame

    def replace_none_with_binary(self, column_names: List[str]):
        for column_name in column_names:
            self.data_frame[column_name] = self.data_frame[column_name].apply(lambda x: 0 if x in [0, 0.0, 'NONE', 'none', 'nan'] else x)
        return self.data_frame

    def convert_bool_to_int(self, column_names: List[str]) -> pd.DataFrame:
        for column_name in column_names:
            self.data_frame[column_name] = self.data_frame[column_name].apply(lambda x: 1 if x in [True, 'true'] else (0 if x in [False, 'false'] else x))
        return self.data_frame
    
    def convert_column_types(self, column_names, new_type):
        for column_name in column_names:
            try:
                if self.data_frame[column_name].dtype == 'object':
                    self.data_frame[column_name] = self.data_frame[column_name].str.strip().replace(' ', '')
                self.data_frame[column_name] = self.data_frame[column_name].astype(new_type)
            except Exception as e:
                print(f"@ {column_name}': {e}")
        return self.data_frame

    def compare_columns(self, col1: str, col2: str) -> None:
        """
        Compares two columns in the DataFrame, displays rows where the values differ,
        and calculates and displays statistics regarding discrepancies.

        Parameters:
        - col1 (str): Name of the first column to compare.
        - col2 (str): Name of the second column to compare.
        """
        # Remove NONE values and compare columns
        df_no_na = self.data_frame.replace('none', pd.NA).dropna(subset=[col1, col2])
        mismatch = df_no_na[df_no_na[col1] != df_no_na[col2]]

        if mismatch.empty:
            print(f"Columns '{col1}' i '{col2}' are identical.")
        else:
            print(f"### {col1} vs {col2} ###")
            for index, row in mismatch.iterrows():
                print(f"Index: {index}, {col1}: {row[col1]}, {col2}: {row[col2]}")

        #Print raport
        column_verify = self.data_frame[col1] == self.data_frame[col2]
        column_verify.dropna(inplace=True)
        total_count = len(column_verify)
        false_count = (~column_verify).sum()
        false_ratio = false_count / total_count if total_count != 0 else float('inf')
        print(f"Boolen values on each row for CONDITION:{col1} == {col2}: {column_verify}")
        print(f"Ratio of false values in CONDITION: {false_ratio}")

    def complete_none(self) -> None:
        self.data_frame.fillna('none', inplace=True)
        self.data_frame.replace([pd.NA, None, 'nan', 'NaN', '', 'NaN ', 'nan '], 'none', inplace=True)
        self.data_frame = self.data_frame.applymap(lambda x: 'none' if pd.isna(x) else x)

    def delete_columns(self, columns: List[str]) -> None:
        self.data_frame.drop(columns, axis=1, inplace=True)

    def unique_values(self, columns: List[str]) -> None:
        for column in columns:
            if column in self.data_frame.columns:
                unique_values = len(self.data_frame[column].unique())
                none_percentage = (self.data_frame[column] == 'none').mean() * 100
                print(f"UNIQUE VALUES FOR '{column}': {unique_values}")
                print(f"Percentage of 'none' values for '{column}': {none_percentage:.2f}%")
                if unique_values < 6:
                    print(f"Unique values in '{column}': {self.data_frame[column].unique()}")
            else:
                print(f"Column {column} does not exist in DataFrame.")


# Data loading and analyzing

## Morizon

In [150]:
morizon = PortalData("morizon")

morizon_df = morizon.load_data().copy(deep=True)

morizon.clean_columns()
morizon.complete_none()
morizon.show_dataframe()

,url,platform,transaction_type,property_type,city,id,total_area,district,prec_adress,row_price,...,material,heating_type,water,elevator,gas,bath_with_wc,electricity,parking,basement,balcony
2920,https://www.morizon.pl/oferta/sprzedaz-mieszka...,morizon,sprzedaz,mieszkanie,warszawa,mzn2043158237,42,ursus,posag 7 panien 16,none,...,none,none,none,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1200,https://www.morizon.pl/oferta/sprzedaz-mieszka...,morizon,sprzedaz,mieszkanie,warszawa,mzn2042982856,81,bialoleka,mankowska,none,...,none,none,none,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1094,https://www.morizon.pl/oferta/sprzedaz-mieszka...,morizon,sprzedaz,mieszkanie,warszawa,mzn2043456202,54,bialoleka,none,770 000,...,none,none,none,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2905,https://www.morizon.pl/oferta/sprzedaz-mieszka...,morizon,sprzedaz,mieszkanie,warszawa,mzn2042976359,35,praga poludnie,podskarbinska 32 34,none,...,none,none,none,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6618,https://www.morizon.pl/oferta/sprzedaz-mieszka...,morizon,sprzedaz,mieszkanie,warszawa,mzn2043483100,61,bemowo,sternicza,903 000,...,none,C.O. Z SIECI MIEJSKIEJ,none,1.0,0.0,0.0,0.0,0.0,1.0,0.0
5575,https://www.morizon.pl/oferta/sprzedaz-mieszka...,morizon,sprzedaz,mieszkanie,warszawa,mzn2043416534,24,mokotow,oskara kolberga,459 000,...,Cegła,Co Miejskie,Woda,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6903,https://www.morizon.pl/oferta/sprzedaz-mieszka...,morizon,sprzedaz,mieszkanie,warszawa,mzn2043443824,74,ursynow,klobucka,1 379 000,...,none,Co Miejskie,Woda (Woda miejska),0.0,0.0,0.0,0.0,1.0,1.0,1.0
1025,https://www.morizon.pl/oferta/sprzedaz-mieszka...,morizon,sprzedaz,mieszkanie,warszawa,mzn2043220103,58,bialoleka,wlodkowica 9,none,...,none,none,none,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7239,https://www.morizon.pl/oferta/sprzedaz-mieszka...,morizon,sprzedaz,mieszkanie,warszawa,mzn2043093468,66,wlochy,jutrzenki szybka,958 464,...,none,none,none,0.0,0.0,0.0,0.0,0.0,0.0,0.0
910,https://www.morizon.pl/oferta/sprzedaz-mieszka...,morizon,sprzedaz,mieszkanie,warszawa,mzn2043326456,47,bialoleka,marywilska,none,...,none,none,none,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Otodom

In [151]:
otodom = PortalData("otodom")

otodom_df = otodom.load_data()

otodom.clean_columns()
otodom.complete_none()
otodom.show_dataframe()

,id,id_realestateagency,date_added,date_update,url,portal_name,market_type,property_type,price,row_price_m2,...,material,heating_type,elevator,heating_type.1,windows,balcony/loggia,parking_type,basement,security,property_rent
6487,65160433,8016959.0,2024-02-29T10:27:55+01:00,2024-03-12T09:16:53+01:00,3-pokojowe-mieszkanie-blisko-toru-sluzewiec-ID...,Otodom,SECONDARY,mieszkanie,1050000.0,16154.0,...,none,none,1.0,none,plastic,1.0,garage,none,"['closed_area', 'entryphone', 'monitoring']",none
1339,65217381,1130589.0,2024-03-14T13:52:47+01:00,2024-03-15T12:52:00+01:00,swietnie-skomunikowane-duzy-balkon-majdanska-I...,Otodom,SECONDARY,mieszkanie,749000.0,14404.0,...,concrete_plate,urban,1.0,urban,none,1.0,none,none,none,none
5991,65149694,89487.0,2024-02-26T19:40:29+01:00,2024-03-04T21:40:29+01:00,metro-ksiecia-janusza-trzy-pokoje-z-balkonem-I...,Otodom,SECONDARY,mieszkanie,689000.0,14035.0,...,none,none,1.0,none,plastic,1.0,none,basement,none,none
6145,65156008,8083484.0,2024-02-28T10:48:36+01:00,2024-03-15T11:40:02+01:00,3-pokojowe-mieszkanie-62m2-2-loggie-bezposredn...,Otodom,PRIMARY,mieszkanie,986236.0,15800.0,...,none,none,1.0,none,none,1.0,garage,none,none,none
8194,65106797,3734815.0,2024-02-15T11:52:03+01:00,2024-02-19T13:21:47+01:00,grzybowska-4-apartament-na-biuro-na-sprzedaz-I...,Otodom,SECONDARY,mieszkanie,4500000.0,33259.0,...,none,none,1.0,none,none,1.0,garage,none,none,1300.0
5981,65111867,none,2024-02-16T12:42:21+01:00,2024-03-10T06:49:54+01:00,warszawa-bemowo-widawska-60m2-3-pokoje-ID4pczp,Otodom,SECONDARY,mieszkanie,829000.0,13817.0,...,other,urban,1.0,urban,plastic,1.0,none,basement,"['anti_burglary_door', 'entryphone', 'monitori...",760.0
1519,65096924,1575662.0,2024-02-13T02:35:09+01:00,2024-03-14T13:39:00+01:00,apartament-137-2-4-pokoje-okopowa-wola-ID4p8Go,Otodom,SECONDARY,mieszkanie,2595000.0,18914.0,...,cellular_concrete,none,1.0,none,none,1.0,garage,none,none,1800.0
4602,64479129,8184254.0,2023-08-03T09:32:25+02:00,2024-03-12T13:02:23+01:00,3-pokojowy-loft-73m2-loggia-bez-posrednikow-ID...,Otodom,PRIMARY,mieszkanie,none,none,...,none,none,1.0,none,none,1.0,garage,none,['closed_area'],none
8934,64964702,2131948.0,2024-01-08T12:16:37+01:00,2024-03-08T12:46:10+01:00,dwupoziomowy-apartament-na-mokotowie-ID4oAhM,Otodom,SECONDARY,mieszkanie,3400000.0,33010.0,...,none,urban,1.0,urban,none,1.0,garage,none,['monitoring'],2800.0
1958,65214149,9658264.0,2024-03-13T18:30:54+01:00,2024-03-13T18:30:57+01:00,oferta-dwustronne-3-pokoje-pierwotny-ID4pDb7,Otodom,PRIMARY,mieszkanie,770000.0,14808.0,...,cellular_concrete,urban,1.0,urban,plastic,1.0,garage,none,"['anti_burglary_door', 'entryphone']",none


### Verification of corresponding columns (with the same name, suffix .1 given by Pandas).

#### district vs district.1

In [152]:
otodom.compare_columns("district", "district.1")

Columns 'district' i 'district.1' are identical.
Boolen values on each row for CONDITION:district == district.1: 0        True
1        True
2        True
3        True
4        True
         ... 
11337    True
11338    True
11339    True
11340    True
11341    True
Length: 11342, dtype: bool
Ratio of false values in CONDITION: 0.0


#### heating_type vs heating_type.1

In [153]:
otodom.compare_columns("heating_type", 'heating_type.1')

Columns 'heating_type' i 'heating_type.1' are identical.
Boolen values on each row for CONDITION:heating_type == heating_type.1: 0        True
1        True
2        True
3        True
4        True
         ... 
11337    True
11338    True
11339    True
11340    True
11341    True
Length: 11342, dtype: bool
Ratio of false values in CONDITION: 0.0


#### street vs street.1

In [154]:
otodom.compare_columns("street", 'street.1')

Columns 'street' i 'street.1' are identical.
Boolen values on each row for CONDITION:street == street.1: 0        True
1        True
2        True
3        True
4        True
         ... 
11337    True
11338    True
11339    True
11340    True
11341    True
Length: 11342, dtype: bool
Ratio of false values in CONDITION: 0.0


##### Removing unnecessary (duplicate) columns from otodom:
    - street.1
    - heating_type.1
    - district.1

In [155]:
duplicated_columns = ['street.1', 'heating_type.1', 'district.1']
otodom.delete_columns(duplicated_columns)

#Check
for column in duplicated_columns: 
    print(column in otodom.df.columns)


False
False
False


## Nieruchomosci Online

In [156]:
no = PortalData("no")
no_df = no.load_data().copy(deep=True)

no.clean_columns()
no.complete_none()
no.show_dataframe()


,balcony/loggia,basement,bulding_type,bulding_year,date_added,date_update,district,electricity,elevator,gas,...,saperate_wc,security,standard,street,total_area,total_property_level,town/city,url,water,windows
5944,True,False,blok mieszkalny,2010,none,20.02.2024,Centrum,True,False,none,...,1.0,"drzwi antywłamaniowe, ochrona",bardzo dobry,Bagno,69.15,14.0,Warszawa,https://warszawa.nieruchomosci-online.pl/miesz...,True,nowe / wymienione
5299,False,False,apartamentowiec,2019,none,14.03.2024,Wola,True,True,none,...,1.0,"domofon/wideofon, drzwi antywłamaniowe, monito...",bardzo dobry,Karolkowa,55.00,8.0,Warszawa,https://warszawa.nieruchomosci-online.pl/miesz...,True,nowe / wymienione
212,True,False,blok mieszkalny,2002,none,18.03.2024,Ursynów,True,True,none,...,1.0,domofon/wideofon,dobry,Lanciego,69.00,10.0,Warszawa,https://warszawa.nieruchomosci-online.pl/miesz...,True,none
3028,False,True,blok mieszkalny,1962,none,14.03.2024,Wola,True,True,none,...,1.0,none,dobry,Chmielna,27.00,10.0,Warszawa,https://warszawa.nieruchomosci-online.pl/miesz...,True,plastikowe
5974,True,False,apartamentowiec,2022,none,27.01.2024,Ursynów,True,True,none,...,none,"drzwi antywłamaniowe, domofon/wideofon, monito...",deweloperski,Kraski,37.00,4.0,Warszawa,https://warszawa.nieruchomosci-online.pl/miesz...,True,none
6486,True,False,apartamentowiec,2017,none,26.02.2024,Praga-Południe,True,False,none,...,none,none,none,Rodziewiczówny,98.00,25.0,Warszawa,https://warszawa.nieruchomosci-online.pl/miesz...,True,nowe / wymienione
2640,True,True,"rok budowy: 2001, technika budowy: tradycyjna/...",2001,none,10.01.2024,Żoliborz,True,True,none,...,2.0,"alarm, domofon/wideofon, monitoring/kamery, oc...",wysoki standard,Zajączka,144.85,5.0,Warszawa,https://warszawa.nieruchomosci-online.pl/miesz...,True,nowe / wymienione
2185,True,False,apartamentowiec,2018,none,30.01.2024,Ochota,True,True,none,...,1.0,none,wysoki standard,Aleje Jerozolimskie,29.00,4.0,Warszawa,https://warszawa.nieruchomosci-online.pl/miesz...,True,none
170,True,False,apartamentowiec,2017,none,18.03.2024,Ursus,True,True,none,...,none,"alarm, domofon/wideofon",dobry,Hennela,59.00,6.0,Warszawa,https://warszawa.nieruchomosci-online.pl/miesz...,True,none
1703,True,False,"rok budowy: 2023 (blok w nowym budownictwie), ...",2023,none,08.11.2023,Gocławek,True,True,none,...,1.0,none,none,Szaserów,75.02,4.0,Warszawa,https://warszawa.nieruchomosci-online.pl/miesz...,True,none


In [157]:
no.compare_columns("no_parking_space", 'parking_type')

### no_parking_space vs parking_type ###
Index: 0, no_parking_space: 1, parking_type: garaż
Index: 1, no_parking_space: 1, parking_type: garaż
Index: 4, no_parking_space: 1, parking_type: w garażu podziemnym (2 miejsca parkingowe)
Index: 5, no_parking_space: 1, parking_type: brak przynależnego miejsca parkingowego
Index: 7, no_parking_space: 1, parking_type: w garażu podziemnym (2 miejsca parkingowe)
Index: 8, no_parking_space: 1, parking_type: przynależne na ulicy (1 miejsce parkingowe)
Index: 9, no_parking_space: 1, parking_type: garaż
Index: 10, no_parking_space: 1, parking_type: garaż wolnostojący (1 miejsce parkingowe)
Index: 12, no_parking_space: 1, parking_type: brak przynależnego miejsca parkingowego
Index: 13, no_parking_space: 1, parking_type: tak
Index: 14, no_parking_space: 1, parking_type: brak przynależnego miejsca parkingowego
Index: 16, no_parking_space: 1, parking_type: przynależne na ulicy (1 miejsce parkingowe)
Index: 18, no_parking_space: 1, parking_type: garaż (2 m

## Define colections for objects

In [158]:
portal_data_objects = [morizon, otodom, no]

objects_dataframes = {
    'morizon': morizon.df,
    'otodom': otodom.df,
    'no': no.df
}

## Proces parking columns in all PortalData instances
Goal: 
- Create column <b>is_parking</b> as [1,0] values if we have information
- Create column <b> parkin_na</b> as [1,0] values, if information was not given (referenced to OtoDom data structure)
- Drop curent columns with parking informations

### N-O

In [159]:
no.df.columns

Index(['balcony/loggia', 'basement', 'bulding_type', 'bulding_year',
       'date_added', 'date_update', 'district', 'electricity', 'elevator',
       'gas', 'heating_type', 'height', 'id_portal', 'id_realestateagency',
       'kitchen_type', 'makret_type', 'material', 'no_parking_space',
       'no_rooms', 'ownership_form', 'parking_type', 'portal_name',
       'prec_adress', 'price', 'property_level', 'property_rent',
       'property_type', 'row_price_m2', 'saperate_wc', 'security', 'standard',
       'street', 'total_area', 'total_property_level', 'town/city', 'url',
       'water', 'windows'],
      dtype='object')

In [160]:
no.unique_values(['parking_type', 'no_parking_space'])

UNIQUE VALUES FOR 'parking_type': 41
Percentage of 'none' values for 'parking_type': 26.29%
UNIQUE VALUES FOR 'no_parking_space': 4
Percentage of 'none' values for 'no_parking_space': 0.00%
Unique values in 'no_parking_space': [1 0 2 3]


In [161]:
no_parking_unique = no.df['parking_type'].unique()
print(no_parking_unique)
len(no_parking_unique)

['garaż' 'none' 'w garażu podziemnym (2 miejsca parkingowe)'
 'brak przynależnego miejsca parkingowego'
 'przynależne na ulicy (1 miejsce parkingowe)'
 'garaż wolnostojący (1 miejsce parkingowe)' 'tak'
 'garaż (2 miejsca parkingowe)'
 'w garażu podziemnym (1 miejsce parkingowe)' 'w garażu podziemnym'
 'przynależne na terenie ogrodzonym (1 miejsce parkingowe)'
 'parking publiczny / na ulicy'
 'przynależne na terenie ogrodzonym (2 miejsca parkingowe)'
 'parking strzeżony w pobliżu, parking publiczny / na ulicy'
 'parking strzeżony w pobliżu'
 'przynależne na ulicy (2 miejsca parkingowe)'
 'garaż w bryle budynku (1 miejsce parkingowe)' 'garaż w bryle budynku'
 'możliwość wykupienia, parking publiczny / na ulicy'
 'przynależne na ulicy' 'możliwość wykupienia'
 'przynależne na terenie ogrodzonym (70 miejsc parkingowych)'
 'przynależne na terenie ogrodzonym'
 'garaż wolnostojący (2 miejsca parkingowe)'
 'garaż wolnostojący (4 miejsca parkingowe)'
 'możliwość wykupienia, parking strzeżony w p

41

In [162]:
#this values gives us information to correct informations given in no_parking_space. 
# If value given, real estate have not law to any parking space or it's additional cost
no_parking_not_own = [
    'brak przynależnego miejsca parkingowego',
    'parking publiczny / na ulicy',
    'możliwość wykupienia',
    'parking strzeżony w pobliżu',
]

no_parking_df = no.df[['parking_type', 'no_parking_space']]

#colect one hot encoding vie of law to parking space with NONEs
no_parking_df['parking_own'] = no_parking_df['parking_type'].apply(
    lambda x: 'none' if x == 'NONE' 
    else (0 
    if any(parking_option in no_parking_not_own for parking_option in str(x).split(',')) 
    else 1)
)
no_parking_df.sample(20)

/var/folders/sh/pxc94xyx7jx4j8wws1vh_np80000gn/T/ipykernel_72353/2730438520.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_parking_df['parking_own'] = no_parking_df['parking_type'].apply(


,parking_type,no_parking_space,parking_own
6180,brak przynależnego miejsca parkingowego,1,0
4887,none,0,1
585,none,0,1
5862,tak,1,1
4078,brak przynależnego miejsca parkingowego,1,0
3069,none,0,1
4549,brak przynależnego miejsca parkingowego,1,0
674,none,0,1
3902,w garażu podziemnym (1 miejsce parkingowe),1,1
6822,none,0,1


In [163]:
no_parking_df.loc[no_parking_df['parking_own'] == 1, 'no_parking_space'] = 1
#colect informations about NONEs to one hot encoding structure
no_parking_df['parking_nan'] = no_parking_df['parking_type'].apply(lambda x: 1 if x == 'none' else 0)
no_parking_df.sample(20)


/var/folders/sh/pxc94xyx7jx4j8wws1vh_np80000gn/T/ipykernel_72353/1364810480.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_parking_df['parking_nan'] = no_parking_df['parking_type'].apply(lambda x: 1 if x == 'none' else 0)


,parking_type,no_parking_space,parking_own,parking_nan
5937,w garażu podziemnym (1 miejsce parkingowe),1,1,0
5238,brak przynależnego miejsca parkingowego,1,0,0
6558,w garażu podziemnym (1 miejsce parkingowe),1,1,0
4287,none,1,1,1
4169,tak,1,1,0
2696,tak,1,1,0
3141,garaż wolnostojący (1 miejsce parkingowe),1,1,0
3170,none,1,1,1
6831,w garażu podziemnym (1 miejsce parkingowe),1,1,0
606,none,1,1,1


In [164]:
#corecting values in 'no_parking_space' by information from 'parking_own' in new col 'is_parking'
no_parking_df['is_parking'] = no_parking_df['no_parking_space'].apply(lambda x: 1 if x != 0 else 0)
no_parking_df.drop(columns=['no_parking_space'], inplace=True)


/var/folders/sh/pxc94xyx7jx4j8wws1vh_np80000gn/T/ipykernel_72353/1273714692.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_parking_df['is_parking'] = no_parking_df['no_parking_space'].apply(lambda x: 1 if x != 0 else 0)
/var/folders/sh/pxc94xyx7jx4j8wws1vh_np80000gn/T/ipykernel_72353/1273714692.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_parking_df.drop(columns=['no_parking_space'], inplace=True)


In [165]:
no.df['is_parking'] = no.df['no_parking_space'].apply(lambda x: 1 if x != 0 else 0)
no.df['parking_nan'] = no.df['parking_type'].apply(lambda x: 1 if x == 'none' else 0)
no.df.drop(columns=['no_parking_space', 'parking_type'], inplace=True)

In [166]:
created_parking_columns = ['is_parking', 'parking_nan']

no.unique_values(created_parking_columns)

UNIQUE VALUES FOR 'is_parking': 2
Percentage of 'none' values for 'is_parking': 0.00%
Unique values in 'is_parking': [1 0]
UNIQUE VALUES FOR 'parking_nan': 2
Percentage of 'none' values for 'parking_nan': 0.00%
Unique values in 'parking_nan': [0 1]


### Otodom

In [167]:
otodom.df.parking_type.unique()

array(['garage', 'none'], dtype=object)

In [168]:
otodom.unique_values(['parking_type'])

UNIQUE VALUES FOR 'parking_type': 2
Percentage of 'none' values for 'parking_type': 38.72%
Unique values in 'parking_type': ['garage' 'none']


In [169]:
otodom_parking_column = "parking_type"

otodom.df['is_parking'] = otodom.df[otodom_parking_column].apply(lambda x: 1 if x not in [None, 'NONE', 'none', 0, '0'] else 0
)
otodom.df['parking_nan'] = otodom.df[otodom_parking_column].apply(lambda x: 1 if x in [None, 'NONE', 'none'] else 0)
otodom.df.drop(columns=[otodom_parking_column], inplace=True)
print(otodom.df[['is_parking', 'parking_nan']].sample(10))

       is_parking  parking_nan
1908            0            1
4288            0            1
2727            0            1
8717            0            1
3768            1            0
2629            0            1
2597            0            1
10810           1            0
5421            1            0
11000           1            0


In [170]:
otodom.unique_values(created_parking_columns)

UNIQUE VALUES FOR 'is_parking': 2
Percentage of 'none' values for 'is_parking': 0.00%
Unique values in 'is_parking': [1 0]
UNIQUE VALUES FOR 'parking_nan': 2
Percentage of 'none' values for 'parking_nan': 0.00%
Unique values in 'parking_nan': [0 1]


### Morizon (same as Otodom)

In [171]:
morizon.df.columns

Index(['url', 'platform', 'transaction_type', 'property_type', 'city', 'id',
       'total_area', 'district', 'prec_adress', 'row_price', 'row_price_m2',
       'primery_market', 'property_level', 'total_property_level',
       'description', 'market_type', 'building_type', 'no_rooms',
       'kitchen_type', 'building_year', 'material', 'heating_type', 'water',
       'elevator', 'gas', 'bath_with_wc', 'electricity', 'parking', 'basement',
       'balcony'],
      dtype='object')

In [172]:
morizon.df.parking.unique()

array([0.0, 1.0, 'none'], dtype=object)

In [173]:
morizon.unique_values(['parking'])

UNIQUE VALUES FOR 'parking': 3
Percentage of 'none' values for 'parking': 0.64%
Unique values in 'parking': [0.0 1.0 'none']


In [174]:
morizon_parking_column = "parking"

morizon.df['parking_nan'] = morizon.df[morizon_parking_column].apply(lambda x: 1 if x == 'none' else 0)
morizon.df['is_parking'] = morizon.df[morizon_parking_column].apply(lambda x: 1 if x not in [None, 'NONE', 'none'] else 0)

morizon.df.drop(columns=[morizon_parking_column], inplace=True)
morizon.df[['is_parking', 'parking_nan']].sample(10)


,is_parking,parking_nan
4494,1,0
1517,1,0
6299,1,0
6793,1,0
4733,1,0
2531,1,0
4270,1,0
3426,1,0
860,1,0
5010,0,1


In [175]:
morizon.unique_values(['is_parking', 'parking_nan'])

UNIQUE VALUES FOR 'is_parking': 2
Percentage of 'none' values for 'is_parking': 0.00%
Unique values in 'is_parking': [1 0]
UNIQUE VALUES FOR 'parking_nan': 2
Percentage of 'none' values for 'parking_nan': 0.00%
Unique values in 'parking_nan': [0 1]


## Match similar column and reneme part of them

### Find same column names and check data

In [176]:
def find_common_columns(*dataframes):
    common_columns = set(dataframes[0].columns)
    for df in dataframes[1:]:
        common_columns.intersection_update(df.columns)
    return list(common_columns)

similar_column_names = find_common_columns(morizon.df, otodom.df, no.df)
similar_column_names

['total_property_level',
 'property_type',
 'material',
 'basement',
 'heating_type',
 'elevator',
 'total_area',
 'parking_nan',
 'property_level',
 'kitchen_type',
 'url',
 'is_parking',
 'row_price_m2',
 'no_rooms',
 'district']

In [177]:
def convert_column_types(df, column_names, new_type):
    for column_name in column_names:
        try:
            if df[column_name].dtype == 'object':
                df[column_name] = df[column_name].str.strip().replace(' ', '')
            df[column_name] = df[column_name].astype(new_type)
        except Exception as e:
            print(f"@ {column_name}': {e}")
    return df

In [178]:
morizon.df[similar_column_names]

,total_property_level,property_type,material,basement,heating_type,elevator,total_area,parking_nan,property_level,kitchen_type,url,is_parking,row_price_m2,no_rooms,district
0,5.0,mieszkanie,none,1.0,CO miejskie,1.0,91,0,4.0,Oddzielna,https://www.morizon.pl/oferta/sprzedaz-mieszka...,1,16 862,4.0,mokotow
1,5.0,mieszkanie,none,0.0,none,1.0,70,0,3.0,none,https://www.morizon.pl/oferta/sprzedaz-mieszka...,1,22 125,3.0,bielany
2,5.0,mieszkanie,none,0.0,none,1.0,110,0,3.0,none,https://www.morizon.pl/oferta/sprzedaz-mieszka...,1,22 125,3.0,ursynow
3,7.0,mieszkanie,none,0.0,Centralne,1.0,76,0,5.0,none,https://www.morizon.pl/oferta/sprzedaz-mieszka...,1,21 484,4.0,mokotow
4,5.0,mieszkanie,Cegła,1.0,Centralne,1.0,86,0,1.0,none,https://www.morizon.pl/oferta/sprzedaz-mieszka...,1,19 302,4.0,ursynow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7303,1.0,mieszkanie,Cegła,1.0,Gazowe,0.0,83,0,1.0,none,https://www.morizon.pl/oferta/sprzedaz-mieszka...,1,16 695,4.0,ochota
7304,4.0,mieszkanie,none,0.0,Co Miejskie,0.0,81,0,1.0,none,https://www.morizon.pl/oferta/sprzedaz-mieszka...,1,17 113,4.0,praga poludnie
7305,4.0,mieszkanie,none,0.0,Co Miejskie,0.0,100,0,1.0,none,https://www.morizon.pl/oferta/sprzedaz-mieszka...,1,17 113,4.0,wlochy
7306,none,mieszkanie,none,0.0,none,0.0,72,0,none,none,https://www.morizon.pl/oferta/sprzedaz-mieszka...,1,none,3.0,zoliborz


In [179]:
otodom.df[similar_column_names]

,total_property_level,property_type,material,basement,heating_type,elevator,total_area,parking_nan,property_level,kitchen_type,url,is_parking,row_price_m2,no_rooms,district
0,7.0,mieszkanie,brick,none,urban,none,79.19,0,['floor_1'],separate kitchen,mieszkanie-79-19-m-warszawa-ID4pzV9,1,16290.0,['3'],Bemowo
1,11.0,mieszkanie,brick,none,urban,1.0,68.14,0,['floor_4'],separate kitchen,3-pokoje-moderna-osiedle-gotowe-do-wprowadzeni...,1,18491.0,['3'],Bródno
2,5.0,mieszkanie,brick,none,electrical,1.0,82.82,1,['floor_4'],none,trzypokojowe-mieszkanie-na-pradze-polnoc-ID4pAwp,0,15500.0,['3'],Targówek
3,3.0,mieszkanie,brick,none,urban,1.0,73.50,0,['floor_2'],none,apartament-w-marinie-3-pokoje-ID4pCuq,1,18367.0,['3'],Żerań
4,4.0,mieszkanie,brick,none,urban,none,70.75,1,['floor_3'],none,mieszkanie-w-sercu-starego-mokotowa-ID4pCJK,0,22332.0,['2'],Mokotów
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11337,4.0,mieszkanie,reinforced_concrete,basement,urban,1.0,67.81,0,['floor_3'],none,3-pok-klobucka-6a-komorka-obok-mieszkania-2gar...,1,16222.0,['2'],Ursynów
11338,5.0,mieszkanie,brick,basement,urban,1.0,89.00,0,['floor_3'],none,apartament-mokotow-chodkiewicza-ID4peUU,1,33767.0,['3'],Górny Mokotów
11339,1.0,mieszkanie,brick,basement,urban,1.0,30.00,0,['floor_10'],separate kitchen,x-ID4p9qQ,1,33333.0,['1'],none
11340,6.0,mieszkanie,brick,none,urban,1.0,46.00,0,['floor_4'],separate kitchen,warszawa-tarchomin-2-pokoje-46m2-po-remoncie-I...,1,15196.0,['2'],Tarchomin


In [180]:
no.df[similar_column_names]

,total_property_level,property_type,material,basement,heating_type,elevator,total_area,parking_nan,property_level,kitchen_type,url,is_parking,row_price_m2,no_rooms,district
0,4.0,Mieszkanie,False,True,miejskie,False,85.00,0,1.0,z oknem,https://warszawa.nieruchomosci-online.pl/miesz...,1,12647.058824,4.0,Ursynów
1,7.0,Mieszkanie,tradycyjna/cegła,False,none,True,106.00,0,2.0,aneks kuchenny,https://warszawa.nieruchomosci-online.pl/miesz...,1,19905.660377,4.0,Mokotów
2,4.0,Mieszkanie,tradycyjna/cegła,True,miejskie,False,55.00,1,0.0,oddzielna,https://warszawa.nieruchomosci-online.pl/miesz...,0,34363.636364,2.0,Śródmieście
3,8.0,Mieszkanie,False,False,miejskie,True,65.52,1,4.0,aneks kuchenny,https://warszawa.nieruchomosci-online.pl/miesz...,0,24267.399267,3.0,Mokotów
4,4.0,Mieszkanie,False,False,none,True,69.76,0,2.0,z oknem,https://warszawa.nieruchomosci-online.pl/miesz...,1,21932.33945,3.0,Wilanów
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7037,3.0,Mieszkanie,False,False,none,True,46.93,0,0.0,"aneks kuchenny, w zabudowie",https://warszawa.nieruchomosci-online.pl/miesz...,1,23226.081398,2.0,Ursynów
7038,2.0,Mieszkanie,False,True,miejskie,False,51.95,0,1.0,aneks kuchenny,https://warszawa.nieruchomosci-online.pl/miesz...,1,15380.173244,2.0,Praga-Południe
7039,3.0,Mieszkanie,False,False,none,True,73.10,1,0.0,aneks kuchenny,https://zamienie.nieruchomosci-online.pl/nowe-...,0,none,4.0,Thomasa Alva Edisona 1
7040,5.0,Mieszkanie,False,False,miejskie,True,58.90,0,3.0,aneks kuchenny,https://warszawa.nieruchomosci-online.pl/miesz...,1,13921.901528,3.0,Białołęka


In [181]:
similar_columns_to_celean_text = [
 'material',
 'no_rooms',
 'elevator',
 'district',
 'property_type',
 'row_price_m2',
 'total_area',
 'kitchen_type',
 'basement',
 'total_property_level',
 'heating_type', 
 'property_level']

for data_frame in portal_data_objects:
    data_frame.clean_text_data(similar_columns_to_celean_text)
    if data_frame.df[similar_columns_to_celean_text].isnull().any().any():
        print("Istnieją wartości NaN w danych - zostały zastąpione na 'NONE")
        data_frame.complete_none()
    

In [182]:
def create_similar_dataframe(column_names_list, **dataframes):
    sorted_columns = sorted(column_names_list)
    
    prefixed_dataframe = pd.DataFrame()

    for col in sorted_columns:
        for platform, df in dataframes.items():
            prefixed_column = f"{platform}_{col}"
            if col in df.columns:
                prefixed_dataframe[prefixed_column] = df[col]
            
    return prefixed_dataframe

sorted_df_with_prefix = create_similar_dataframe(similar_column_names, morizon=morizon.df, otodom=otodom.df, no=no.df)
sorted_df_with_prefix.columns


Index(['morizon_basement', 'otodom_basement', 'no_basement',
       'morizon_district', 'otodom_district', 'no_district',
       'morizon_elevator', 'otodom_elevator', 'no_elevator',
       'morizon_heating_type', 'otodom_heating_type', 'no_heating_type',
       'morizon_is_parking', 'otodom_is_parking', 'no_is_parking',
       'morizon_kitchen_type', 'otodom_kitchen_type', 'no_kitchen_type',
       'morizon_material', 'otodom_material', 'no_material',
       'morizon_no_rooms', 'otodom_no_rooms', 'no_no_rooms',
       'morizon_parking_nan', 'otodom_parking_nan', 'no_parking_nan',
       'morizon_property_level', 'otodom_property_level', 'no_property_level',
       'morizon_property_type', 'otodom_property_type', 'no_property_type',
       'morizon_row_price_m2', 'otodom_row_price_m2', 'no_row_price_m2',
       'morizon_total_area', 'otodom_total_area', 'no_total_area',
       'morizon_total_property_level', 'otodom_total_property_level',
       'no_total_property_level', 'morizon_url'

In [183]:
similar_columns = {
#numerical
'row_price_m2', #predicted value / decision atribute is price
'no_rooms',
'property_level',
'total_area',
'total_property_level',

#True / False
'basement',
'elevator',

#Categorical
'heating_type',
'kitchen_type',
'material',
'property_type',
'district',

#Not checking
'parking_nan',
'is_parking',
'url'}

In [184]:
def display_sample_with_columns_list(dataframe, columns):
    filtered_columns = [col for col in dataframe.columns if any(substring in col for substring in columns)]
    filtered_df = dataframe[filtered_columns]
    if not filtered_df.empty:
        for col in filtered_df.columns:
            unique_values = len(filtered_df[col].unique())
            none_percentage = (filtered_df[col] == 'none').mean() * 100
            print(f"UNIQUE VALUES FOR '{col}': {unique_values}")
            print(f"Percentage of 'none' values for '{col}': {none_percentage:.2f}%")
            if unique_values < 6:
                print(f"Unique values in '{col}': {filtered_df[col].unique()}")
        return filtered_df.sample(6)
    else:
        return pd.DataFrame(columns=filtered_columns).sample(6)


In [185]:
def generate_column_value_dataframe(portal_data_objects, column_name, value_type=None):
    # Creating new dataframe with given column

    column_dfs = [obj.df[[column_name]] for obj in portal_data_objects if column_name in obj.df.columns]
    
    # Combining all dataframes into one
    combined_df = pd.concat(column_dfs, ignore_index=True)
    
    # Calculating the percentage of unique values
    value_counts = combined_df[column_name].value_counts(normalize=True) * 100
    
    # Rounding to 1 decimal place and adding %
    value_counts = value_counts.round(1).astype(str) + '%'
    
    # Displaying results
    if value_type:
        value_counts = value_counts[value_counts.index.map(lambda x: isinstance(x, value_type))]
    
    print(value_counts)

##### row_price_m2
- morizon: NONE > 33%

This is a decision-related argument, hence its analysis will take place at the very beginning, due to the potential removal of some rows. In the next section, a more extensive analysis of the PRICE / ROW_PRICE column will take place after renaming the arguments

In [186]:
display_sample_with_columns_list(sorted_df_with_prefix, ['row_price_m2'])

UNIQUE VALUES FOR 'morizon_row_price_m2': 2698
Percentage of 'none' values for 'morizon_row_price_m2': 33.68%
UNIQUE VALUES FOR 'otodom_row_price_m2': 4155
Percentage of 'none' values for 'otodom_row_price_m2': 6.44%
UNIQUE VALUES FOR 'no_row_price_m2': 4734
Percentage of 'none' values for 'no_row_price_m2': 1.01%


,morizon_row_price_m2,otodom_row_price_m2,no_row_price_m2
6083,none,16001.0,16350.7109
3575,26 875,23200.0,17995.689655
2902,none,31364.0,19459.459459
2369,14 180,23226.0,15000.0
2668,none,none,23513.234789
2623,19 213,19459.0,28814.814815


##### Morizon

In [187]:
morizon.df[morizon.df['row_price_m2'] == 'none'][['row_price_m2', 'row_price', 'total_area', 'market_type', 'primery_market']]


,row_price_m2,row_price,total_area,market_type,primery_market
5,none,none,117,pierwotny,none
6,none,none,75,pierwotny,none
7,none,none,119,pierwotny,none
8,none,none,89,pierwotny,none
13,none,none,66,pierwotny,none
...,...,...,...,...,...
7236,none,none,66,pierwotny,none
7264,none,none,74,pierwotny,none
7296,none,none,100,none,none
7306,none,none,72,pierwotny,none


In [188]:
morizon.df[morizon.df['row_price'] != morizon.df['primery_market']]

,url,platform,transaction_type,property_type,city,id,total_area,district,prec_adress,row_price,...,heating_type,water,elevator,gas,bath_with_wc,electricity,basement,balcony,parking_nan,is_parking


In [189]:
morizon.df[(morizon.df['row_price_m2'] == 'none') & (morizon.df['market_type'] != 'pierwotny')][['row_price_m2', 'row_price', 'total_area', 'market_type', 'primery_market']]


,row_price_m2,row_price,total_area,market_type,primery_market
1130,none,none,61,none,none
1131,none,none,61,none,none
1138,none,none,51,none,none
1139,none,none,51,none,none
1143,none,none,50,none,none
1144,none,none,50,none,none
1385,none,none,60,none,none
1386,none,none,60,none,none
1392,none,none,57,none,none
1393,none,none,57,none,none


For morizon, if row_price_m2 == 'NONE':

- in about 50 cases the price and market type is NONE
- for about 2400 cases is unknown 

For real estate price prediction row_price_2 or row_price will be key, hence rows are unnecessary -> 33% of dataframe morizon

In [190]:
otodom.df[(otodom.df['row_price_m2'] == 'none')][['row_price_m2', 'price', 'total_area', 'market_type']]

,row_price_m2,price,total_area,market_type
20,none,none,31.33,PRIMARY
22,none,none,42.67,PRIMARY
86,none,none,69.63,PRIMARY
88,none,none,38.69,PRIMARY
92,none,none,73.62,PRIMARY
...,...,...,...,...
11269,none,none,46.81,PRIMARY
11275,none,none,53.64,PRIMARY
11276,none,none,47.18,PRIMARY
11287,none,none,53.02,PRIMARY


In [191]:
otodom.df[(otodom.df['row_price_m2'] == 'none') & (otodom.df['market_type'] != 'PRIMARY')][['row_price_m2', 'price', 'total_area', 'market_type']]

,row_price_m2,price,total_area,market_type


In [192]:
no.df[(no.df['row_price_m2'] == 'none')][['row_price_m2', 'price', 'total_area', 'makret_type']]

,row_price_m2,price,total_area,makret_type
1544,none,none,26.00,none
6647,none,none,75.00,pierwotny
6648,none,none,75.00,pierwotny
6653,none,none,66.08,pierwotny
6656,none,none,54.12,pierwotny
...,...,...,...,...
7032,none,none,64.00,pierwotny
7033,none,none,62.51,pierwotny
7034,none,none,72.15,pierwotny
7035,none,none,75.05,pierwotny


In [193]:
no.df[(no.df['row_price_m2'] == 'none') & (no.df['makret_type'] != 'pierwotny')][['row_price_m2', 'price', 'total_area', 'makret_type']]

,row_price_m2,price,total_area,makret_type
1544,none,none,26.0,none


In [194]:
for data_frame in portal_data_objects:
    data_frame.df.drop(data_frame.df[data_frame.df['row_price_m2'] == 'none'].index, inplace=True)
    # Removing all spaces from strings
    data_frame.df['row_price_m2'] = data_frame.df['row_price_m2'].astype(str).str.replace(' ', '').astype(float).astype(int)
    data_frame.convert_column_types(['row_price_m2'], int)
    data_frame.unique_values(['row_price_m2'])

UNIQUE VALUES FOR 'row_price_m2': 2697
Percentage of 'none' values for 'row_price_m2': 0.00%
UNIQUE VALUES FOR 'row_price_m2': 5602
Percentage of 'none' values for 'row_price_m2': 0.00%
UNIQUE VALUES FOR 'row_price_m2': 4123
Percentage of 'none' values for 'row_price_m2': 0.00%


##### no_rooms
- Otodom, change ['3'], ['5'] format for 3, 5 
- Otodom, change ['more', 'NONE', {>8 values} for 100]

All -> change for int not float

In [195]:
display_sample_with_columns_list(sorted_df_with_prefix, ['no_rooms'])

UNIQUE VALUES FOR 'morizon_no_rooms': 11
Percentage of 'none' values for 'morizon_no_rooms': 0.67%
UNIQUE VALUES FOR 'otodom_no_rooms': 10
Percentage of 'none' values for 'otodom_no_rooms': 0.00%
UNIQUE VALUES FOR 'no_no_rooms': 16
Percentage of 'none' values for 'no_no_rooms': 0.34%


,morizon_no_rooms,otodom_no_rooms,no_no_rooms
5919,2.0,2,5.0
7221,4.0,2,NaN
5697,4.0,4,2.0
1497,2.0,2,3.0
2817,4.0,5,3.0
1721,1.0,2,3.0


In [196]:
otodom.df['no_rooms'].unique()

array(['3', '2', '4', '1', '5', '8', '6', '7', 'more', '9', '10'],
      dtype=object)

In [197]:
generate_column_value_dataframe(portal_data_objects, 'no_rooms', str)

no_rooms
3       17.0%
2       16.8%
4        6.3%
1        4.4%
5        1.8%
6        0.3%
none     0.1%
7        0.1%
more     0.1%
8        0.0%
10       0.0%
9        0.0%
Name: proportion, dtype: object


In [198]:
max_number_of_rooms = 9

for data_frame in portal_data_objects:
    data_frame.df = data_frame.df.astype(str)
    data_frame.df['no_rooms'] = data_frame.df['no_rooms'].replace('none', '100')
    data_frame.df['no_rooms'] = data_frame.df['no_rooms'].replace('more', '100')  # Replace 'none' with 100
    # Conversion of the number of rooms to int type, with special cases
    data_frame.df['no_rooms'] = data_frame.df['no_rooms'].apply(lambda x: int(x))
    data_frame.df['no_rooms'] = data_frame.df['no_rooms'].apply(lambda x: 100 if x > max_number_of_rooms else x)

generate_column_value_dataframe(portal_data_objects, 'no_rooms')


no_rooms
3      36.3%
2      36.2%
4      13.1%
1       9.8%
5       3.4%
6       0.6%
100     0.2%
7       0.2%
8       0.1%
9       0.0%
Name: proportion, dtype: object


##### property level AND total_property_level
- otodom: ['floor_3']	== 3, ['ground_floor']	== 0
- morizon: NONE for property_level == total_property_level, primery_market? NOT ALWAYS
- all platfroms -> int()

In [199]:
display_sample_with_columns_list(sorted_df_with_prefix, ['property_level'])

UNIQUE VALUES FOR 'morizon_property_level': 23
Percentage of 'none' values for 'morizon_property_level': 23.75%
UNIQUE VALUES FOR 'otodom_property_level': 15
Percentage of 'none' values for 'otodom_property_level': 2.26%
UNIQUE VALUES FOR 'no_property_level': 32
Percentage of 'none' values for 'no_property_level': 0.66%
UNIQUE VALUES FOR 'morizon_total_property_level': 30
Percentage of 'none' values for 'morizon_total_property_level': 23.75%
UNIQUE VALUES FOR 'otodom_total_property_level': 32
Percentage of 'none' values for 'otodom_total_property_level': 3.05%
UNIQUE VALUES FOR 'no_total_property_level': 35
Percentage of 'none' values for 'no_total_property_level': 3.00%


,morizon_property_level,otodom_property_level,no_property_level,morizon_total_property_level,otodom_total_property_level,no_total_property_level
2207,none,floor 1,2.0,none,4.0,4.0
3097,none,ground floor,1.0,none,2.0,4.0
5891,none,floor 1,3.0,none,4.0,4.0
739,1.0,floor 2,2.0,4.0,8.0,3.0
6517,2.0,floor 1,7.0,2.0,3.0,7.0
3374,none,none,12.0,none,10.0,12.0


###### otodom specyfic

In [200]:
otodom.df['property_level'].unique()

array(['floor 1', 'floor 4', 'floor 2', 'floor 3', 'ground floor', 'none',
       'floor 5', 'floor 7', 'floor 10', 'floor 6', 'floor 9',
       'floor higher 10', 'floor 8', 'cellar', 'garret'], dtype=object)

In [201]:
otodom.df['property_level'] = otodom.df['property_level'].replace("ground floor", 0)
otodom.df['property_level'] = otodom.df['property_level'].replace("cellar", 111) #for string transformation
otodom.df['property_level'] = otodom.df['property_level'].replace("floor higher 10", 100)

otodom.df['property_level'] = otodom.df.apply(lambda row: row['total_property_level'] if row['property_level'] == "garret" else row['property_level'], axis=1)

otodom.df['property_level'].unique()

array(['floor 1', 'floor 4', 'floor 2', 'floor 3', 0, 'none', 'floor 5',
       'floor 7', 'floor 10', 'floor 6', 'floor 9', 100, 'floor 8', 111,
       11.0, 52.0, 2.0, 10.0], dtype=object)

In [202]:
otodom.df['property_level'] = otodom.df['property_level'].astype(str).str.extract('(\d+)')
otodom.df['property_level'] = otodom.df['property_level'].astype(float)
otodom.df['property_level'] = otodom.df['property_level'].replace(111, -1) #after string transformation
otodom.df['property_level'] = otodom.df['property_level'].replace({np.nan: 'none'}) #same as in other columns

otodom.df['property_level'].unique() #ground floor, cellar, garret, floor higher 10

array([1.0, 4.0, 2.0, 3.0, 0.0, 'none', 5.0, 7.0, 10.0, 6.0, 9.0, 100.0,
       8.0, -1.0, 11.0, 52.0], dtype=object)

In [203]:
generate_column_value_dataframe(portal_data_objects, 'property_level', value_type=str)


property_level
none    4.9%
Name: proportion, dtype: object


In [204]:
otodom.df[otodom.df['property_level'] == 'none'][['property_level', 'total_property_level']]


,property_level,total_property_level
8,none,3.0
29,none,none
32,none,4.0
368,none,6.0
414,none,6.0
...,...,...
10897,none,none
10910,none,1.0
11232,none,1.0
11233,none,1.0


###### morizon specyfic

In [205]:
morizon.df[morizon.df['property_level'] == 'none'][['property_level', 'total_property_level', 'market_type']]


,property_level,total_property_level,market_type
22,none,none,wtorny
23,none,none,wtorny
25,none,none,wtorny
50,none,none,wtorny
51,none,none,wtorny
...,...,...,...
7283,none,none,pierwotny
7289,none,none,pierwotny
7290,none,none,pierwotny
7295,none,none,pierwotny


In [206]:
print(morizon.df[(morizon.df['property_level'] == 'none') & (morizon.df['total_property_level'] != 'none')][['property_level', 'total_property_level', 'market_type']])


Empty DataFrame
Columns: [property_level, total_property_level, market_type]
Index: []


In [207]:
unique_market_type_percentage = morizon.df[morizon.df['property_level'] == 'none']['market_type'].value_counts(normalize=True) * 100
display(unique_market_type_percentage)

market_type
wtorny       75.126904
pierwotny    24.873096
Name: proportion, dtype: float64

For Morizon:
- if property_level == 'NONE', total_property_level == 'NONE'
- market_type has no relation to the above condition

In [208]:
for data_frame in portal_data_objects:
    data_frame.create_nan_column(['property_level', 'total_property_level'])
    
    data_frame.df['property_level'] = data_frame.df['property_level'].apply(lambda x: 100 if x == 'none' else x).astype(int)
    data_frame.df['total_property_level'] = data_frame.df['total_property_level'].apply(lambda x: 100 if x == 'none' else x).astype(int)
    
    print(f"property_level: {data_frame.df['property_level'].unique()}")
    print(f"total_property_level: {data_frame.df['total_property_level'].unique()}")

property_level: [  4   3   5   1   2 100   6  10   9   7  14   8  12  17  11  15  22  13
  20  16  23  18  24]
total_property_level: [  5   7   1   3   4   2 100   9  10   6  12  14  15  11   8  13  17  19
  26  16  22  18  24  44  25  23  30  20  21  28]
property_level: [  1   4   2   3   0 100   5   7  10   6   9   8  -1  11  52]
total_property_level: [   7   11    5    3    4    2    9   12    8   10  100   15    6    1
   52   14   30   21   13   16   18   17   25   22   27   23   20   24
   19   29   28   31   26   44 2019  106]
property_level: [  1   2   0   4   5   3  10   6   7  11   8   9  18 100  19  17  15  20
  12  14  16  24  28  13  31  52  23  51  22  29  30]
total_property_level: [  4   7   8   2   6   5   1  10   3 100  14  15  11  13  12  18  17   9
  16  30  23  20  25  24  22  19  26  44  29  28  52  27  21 106]


In [209]:
# Code for fixing errors - at the visualization stage before the model, not cleaning data - for identification of dependencies in the entire set
# Changing values greater than 46 to 100 in specific columns for all DataFrame objects in portal_data_objects
highest_level_cut = 21
property_level_columns = ['property_level', 'total_property_level']

for data_frame in portal_data_objects:
    for column in property_level_columns:
        data_frame.df[column] = data_frame.df[column].astype(int)
        #change outliers to 100 (uknnown)
        data_frame.df[column] = data_frame.df[column].apply(lambda x: 100 if x > highest_level_cut else x)
        data_frame.convert_column_types(property_level_columns, float)

In [210]:
generate_column_value_dataframe(portal_data_objects, 'property_level')

property_level
 1.0      19.6%
 2.0      17.7%
 3.0      14.8%
 4.0      11.1%
 0.0      10.6%
 5.0       6.0%
 100.0     5.9%
 6.0       4.6%
 7.0       3.3%
 8.0       2.3%
 9.0       1.6%
 10.0      1.4%
 11.0      0.3%
 12.0      0.2%
 14.0      0.1%
 17.0      0.1%
 13.0      0.1%
 15.0      0.1%
 16.0      0.1%
 20.0      0.0%
 18.0      0.0%
-1.0       0.0%
 19.0      0.0%
Name: proportion, dtype: object


In [211]:
generate_column_value_dataframe(portal_data_objects, 'total_property_level')

total_property_level
4.0      18.9%
3.0      15.0%
5.0      11.0%
10.0      8.3%
7.0       8.0%
6.0       7.5%
100.0     6.8%
2.0       6.7%
8.0       6.2%
11.0      2.6%
9.0       2.3%
12.0      1.7%
15.0      1.6%
1.0       1.1%
13.0      0.6%
16.0      0.5%
17.0      0.5%
14.0      0.4%
18.0      0.1%
19.0      0.1%
20.0      0.1%
21.0      0.0%
Name: proportion, dtype: object


##### total_area
- all platforms -> float()

In [212]:
display_sample_with_columns_list(sorted_df_with_prefix, ['total_area'])

UNIQUE VALUES FOR 'morizon_total_area': 197
Percentage of 'none' values for 'morizon_total_area': 0.00%
UNIQUE VALUES FOR 'otodom_total_area': 2572
Percentage of 'none' values for 'otodom_total_area': 0.00%
UNIQUE VALUES FOR 'no_total_area': 2137
Percentage of 'none' values for 'no_total_area': 0.00%


,morizon_total_area,otodom_total_area,no_total_area
1075,71,65.18,67.80
5500,113,54.93,155.00
7139,42,62.00,NaN
2492,86,48.50,86.17
6606,103,38.00,36.22
6546,56,49.40,69.00


In [213]:
total_area_min_cut = 18
total_area_max_cut = 195

for data_frame in portal_data_objects:
    data_frame.convert_column_types(['total_area'], float)
    print(f"Number of rows: {len(data_frame.df['total_area'])}")
    print("Smallest values of total_area (from 15. index): ", sorted(data_frame.df['total_area'].unique())[15:25])
    print("Largest values of total_area (from 20. index): ", sorted(data_frame.df['total_area'].unique(), reverse=True)[20:30])
    print(f"Percentage of observations with 'total_area' < {total_area_min_cut}: {round((data_frame.df['total_area'] < total_area_min_cut).sum() / len(data_frame.df) * 100, 2)}%")
    print(f"Percentage of observations with 'total_area' > {total_area_max_cut}: {round((data_frame.df['total_area'] > total_area_max_cut).sum() / len(data_frame.df) * 100, 2)}%\n")


Number of rows: 4847
Smallest values of total_area (from 15. index):  [29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0]
Largest values of total_area (from 20. index):  [224.0, 222.0, 220.0, 213.0, 205.0, 203.0, 200.0, 197.0, 196.0, 195.0]
Percentage of observations with 'total_area' < 18: 0.21%
Percentage of observations with 'total_area' > 195: 0.7%

Number of rows: 10377
Smallest values of total_area (from 15. index):  [16.24, 16.25, 16.26, 17.0, 17.26, 17.29, 17.5, 17.57, 17.7, 18.0]
Largest values of total_area (from 20. index):  [347.7, 336.0, 333.2, 333.19, 319.0, 317.0, 316.0, 303.0, 300.34, 300.3]
Percentage of observations with 'total_area' < 18: 0.32%
Percentage of observations with 'total_area' > 195: 1.37%

Number of rows: 6968
Smallest values of total_area (from 15. index):  [18.44, 18.62, 19.0, 20.0, 20.11, 20.13, 20.66, 20.82, 20.9, 21.0]
Largest values of total_area (from 20. index):  [274.21, 274.03, 274.0, 271.0, 267.0, 262.0, 254.38, 254.0, 252.0, 250.0]
P

In [214]:
for data_frame in portal_data_objects:
    print(data_frame.df["total_area"].dtype)

    data_frame.df.drop(data_frame.df[data_frame.df['total_area'] <total_area_min_cut ].index, inplace=True)
    data_frame.df.drop(data_frame.df[data_frame.df['total_area'] > total_area_max_cut ].index, inplace=True)

    print(f"Number of rows: {len(data_frame.df['total_area'])}")
    print(f"Percentage of observations (after transformation) with 'total_area' < {total_area_min_cut}: {round((data_frame.df['total_area'] < total_area_min_cut).sum() / len(data_frame.df) * 100, 2)}%")
    print(f"Percentage of observations (after transformation) with 'total_area' > {total_area_max_cut}: {round((data_frame.df['total_area'] > total_area_max_cut).sum() / len(data_frame.df) * 100, 2)}%\n")



float64
Number of rows: 4803
Percentage of observations (after transformation) with 'total_area' < 18: 0.0%
Percentage of observations (after transformation) with 'total_area' > 195: 0.0%

float64
Number of rows: 10202
Percentage of observations (after transformation) with 'total_area' < 18: 0.0%
Percentage of observations (after transformation) with 'total_area' > 195: 0.0%

float64
Number of rows: 6876
Percentage of observations (after transformation) with 'total_area' < 18: 0.0%
Percentage of observations (after transformation) with 'total_area' > 195: 0.0%



#####  basement:
- OTODOM: change 'basement' in basement == 1, NONE == 0
- NO: change True == 1, False == 0

For all platrofms:
if NONE in basement:
- NONE = 0
- basement_nan == 1 (else 0)

!!! Together with elevator below

In [215]:
display_sample_with_columns_list(sorted_df_with_prefix, ['basement'])

UNIQUE VALUES FOR 'morizon_basement': 3
Percentage of 'none' values for 'morizon_basement': 0.64%
Unique values in 'morizon_basement': [1.0 0.0 'none']
UNIQUE VALUES FOR 'otodom_basement': 2
Percentage of 'none' values for 'otodom_basement': 68.25%
Unique values in 'otodom_basement': ['none' 'basement']
UNIQUE VALUES FOR 'no_basement': 4
Percentage of 'none' values for 'no_basement': 23.13%
Unique values in 'no_basement': [True False 'none' nan]


,morizon_basement,otodom_basement,no_basement
723,0.0,none,True
1416,0.0,none,True
2565,1.0,none,False
6494,1.0,none,False
4976,0.0,basement,True
6981,0.0,basement,True


#####  elevator
- Otodom: NONE == 0
- NO: True == 1, False == 0, 

In [216]:
display_sample_with_columns_list(sorted_df_with_prefix, ['elevator'])

UNIQUE VALUES FOR 'morizon_elevator': 3
Percentage of 'none' values for 'morizon_elevator': 0.64%
Unique values in 'morizon_elevator': [1.0 0.0 'none']
UNIQUE VALUES FOR 'otodom_elevator': 2
Percentage of 'none' values for 'otodom_elevator': 29.63%
Unique values in 'otodom_elevator': ['none' 1.0]
UNIQUE VALUES FOR 'no_elevator': 3
Percentage of 'none' values for 'no_elevator': 0.00%
Unique values in 'no_elevator': [False True nan]


,morizon_elevator,otodom_elevator,no_elevator
955,0.0,1.0,False
238,0.0,1.0,False
302,0.0,1.0,False
2418,0.0,1.0,False
695,1.0,1.0,False
7000,1.0,1.0,True


In [217]:
binary_columns = ['elevator', 'basement']


for data_frame in portal_data_objects:
    data_frame.create_nan_column(binary_columns)
    if data_frame == no:
        data_frame.convert_bool_to_int(binary_columns)
    elif data_frame == otodom:
        otodom.df['basement'] = otodom.df['basement'].apply(lambda x: 1 if x == 'basement' else 0)
    data_frame.replace_none_with_binary(binary_columns)
    data_frame.convert_column_types(binary_columns, int)
    data_frame.unique_values(binary_columns)

UNIQUE VALUES FOR 'elevator': 2
Percentage of 'none' values for 'elevator': 0.00%
Unique values in 'elevator': [1 0]
UNIQUE VALUES FOR 'basement': 2
Percentage of 'none' values for 'basement': 0.00%
Unique values in 'basement': [1 0]
UNIQUE VALUES FOR 'elevator': 2
Percentage of 'none' values for 'elevator': 0.00%
Unique values in 'elevator': [0 1]
UNIQUE VALUES FOR 'basement': 2
Percentage of 'none' values for 'basement': 0.00%
Unique values in 'basement': [0 1]
UNIQUE VALUES FOR 'elevator': 2
Percentage of 'none' values for 'elevator': 0.00%
Unique values in 'elevator': [0 1]
UNIQUE VALUES FOR 'basement': 2
Percentage of 'none' values for 'basement': 0.00%
Unique values in 'basement': [1 0]


In [218]:
otodom.df.basement.unique()

array([0, 1])

##### heating_type
- Potentialy 33 cases (Morizon)
- NONE values on very high level
- other dictionaries. 

To analyse - if used, heating_nan needed for NO

In [219]:
display_sample_with_columns_list(sorted_df_with_prefix, ['heating_type'])

UNIQUE VALUES FOR 'morizon_heating_type': 34
Percentage of 'none' values for 'morizon_heating_type': 56.87%
UNIQUE VALUES FOR 'otodom_heating_type': 7
Percentage of 'none' values for 'otodom_heating_type': 27.79%
UNIQUE VALUES FOR 'no_heating_type': 14
Percentage of 'none' values for 'no_heating_type': 49.49%


,morizon_heating_type,otodom_heating_type,no_heating_type
2910,none,urban,none
2516,ogrzewanie miejskie,gas,none
144,co miejskie,none,miejskie
753,c o z sieci miejskiej,none,none
660,c o z sieci miejskiej,none,none
2372,none,urban,none


In [220]:
morizon.create_nan_column(['heating_type'])
morizon.df['heating_type'] = morizon.df['heating_type'].str.lower().str.strip()
morizon.df['heating_type'].unique()

array(['co miejskie', 'none', 'centralne', 'gazowe', 'co własne',
       'miejskie', 'c o z sieci miejskiej', 'ogrzewanie miejskie',
       'c o miejskie', 'elektryczne', 'c o własne', 'c o gazowe',
       'co z własnej kotłowni', 'pompa ciepła',
       'ogrzewanie na gaz miejski', 'c o gazowe inne',
       'c o gazowe c o własne', 'c o lokalne', 'ekologiczne', 'piec',
       'piec pompa ciepła', 'inne', 'c o gazowe piec',
       'ogrzewanie elektryczne', 'c o elektryczne', 'własne gazowe',
       'z własnej kotłowni', 'eta owe', 'c o elektryczne c o gazowe',
       'zroznicowane', 'własne dla budynku', 'własne gazowe junkers',
       'gazowe budynkowe'], dtype=object)

In [221]:
otodom.create_nan_column(['heating_type'])
otodom.df['heating_type'] = otodom.df['heating_type'].str.lower().str.strip()
otodom.df['heating_type'].unique()

array(['urban', 'electrical', 'none', 'other', 'gas', 'boiler room',
       'tiled stove'], dtype=object)

In [222]:
no.create_nan_column(['heating_type'])
no.df['heating_type'] = no.df['heating_type'].str.lower().str.strip()
no.df['heating_type'].unique()

array(['miejskie', 'none', 'gazowe', 'centralne ogrzewanie', 'własne',
       'inne', 'elektryczne', 'pompa ciepła', 'piec', 'olej opałowy',
       'co z własnej kotłowni', 'kominek', 'podłogowe'], dtype=object)

In [223]:
generate_column_value_dataframe(portal_data_objects, 'heating_type', str)

heating_type
none                          36.4%
urban                         29.7%
miejskie                      16.0%
co miejskie                    5.3%
c o z sieci miejskiej          4.2%
gas                            1.9%
other                          1.8%
gazowe                         1.3%
c o miejskie                   0.5%
boiler room                    0.4%
własne                         0.3%
ogrzewanie miejskie            0.3%
centralne                      0.3%
c o gazowe                     0.3%
centralne ogrzewanie           0.3%
electrical                     0.2%
co własne                      0.1%
c o własne                     0.1%
pompa ciepła                   0.1%
elektryczne                    0.1%
inne                           0.1%
ogrzewanie na gaz miejski      0.1%
co z własnej kotłowni          0.0%
c o gazowe c o własne          0.0%
piec                           0.0%
własne gazowe                  0.0%
tiled stove                    0.0%
c o elektryczne

In [224]:
heating_types_dict = {
    "urban": [
        'miejskie', 'c o miejskie', 'c o z sieci miejskiej', 'ogrzewanie miejskie', 'co miejskie'
    ],
    "central": [
        'centralne', 'centralne ogrzewanie', 'c o', 'c o elektryczne c o z sieci miejskiej kominek'
    ],
    "gas": [
        'gazowe', 'c o gazowe', 'ogrzewanie na gaz miejski', 'c o gazowe inne', 'c o gazowe piec', 'własne gazowe', 'gazowe budynkowe'
    ],
    "individual": [
        'własne', 'co własne', 'c o własne', 'własne dla budynku', 'co z własnej kotłowni', 'c o z własnej kotłowni', 'c o lokalne', 'boiler room', 'z własnej kotłowni'
    ],
    "electrical": [
        'elektryczne', 'ogrzewanie elektryczne', 'c o elektryczne'
    ],
    "heat_pump": [
        'pompa ciepła', 'piec pompa ciepła'
    ],
    "fireplace": [
        'kominek'
    ],
    "stove": [
        'piec', 'tiled stove'
    ],
    "other": [
        'inne', 'zroznicowane', 'ekologiczne', 'eta owe', 'eta?owe'
    ],
    "oil": [
        'olej opałowy'
    ],
    "underfloor": [
        'podłogowe'
    ],
    "UNKNOWN": [
        'none'
    ]
}


def replace_values_with_keys(portal_data_objects, column_name, replace_dict):
    for obj in portal_data_objects:
        for key, terms in replace_dict.items():
            for term in terms:
                obj.df[column_name] = obj.df[column_name].replace(term, key)

for data_frame in portal_data_objects:
    data_frame.create_nan_column(['heating_type'])

replace_values_with_keys(portal_data_objects, 'heating_type', heating_types_dict)

morizon.df['heating_type'] = morizon.df['heating_type'].apply(lambda x: 'gas' if 'gaz' in x else x)
morizon.df['heating_type'] = morizon.df['heating_type'].apply(lambda x: 'urban' if 'c.o.' in x else x)

In [225]:
generate_column_value_dataframe(portal_data_objects, 'heating_type')

heating_type
urban         56.1%
UNKNOWN       36.4%
gas            3.6%
other          1.9%
individual     1.1%
central        0.6%
electrical     0.3%
heat_pump      0.1%
stove          0.0%
oil            0.0%
fireplace      0.0%
underfloor     0.0%
Name: proportion, dtype: object


In [226]:
heating_normilize_dict = {
    'other': ['heat_pump', 'stove', 'tiled_stove', 'oil', 'fireplace', 'underfloor']
}

replace_values_with_keys(portal_data_objects, 'heating_type', heating_normilize_dict)
generate_column_value_dataframe(portal_data_objects, 'heating_type')

heating_type
urban         56.1%
UNKNOWN       36.4%
gas            3.6%
other          2.0%
individual     1.1%
central        0.6%
electrical     0.3%
Name: proportion, dtype: object


##### kitchen_type -> DROP COLUMN
Morizon: 75% NONE
Otodom: 81%




In [227]:
display_sample_with_columns_list(sorted_df_with_prefix, ['kitchen_type'])

UNIQUE VALUES FOR 'morizon_kitchen_type': 10
Percentage of 'none' values for 'morizon_kitchen_type': 75.94%
UNIQUE VALUES FOR 'otodom_kitchen_type': 2
Percentage of 'none' values for 'otodom_kitchen_type': 81.06%
Unique values in 'otodom_kitchen_type': ['separate kitchen' 'none']
UNIQUE VALUES FOR 'no_kitchen_type': 21
Percentage of 'none' values for 'no_kitchen_type': 21.13%


,morizon_kitchen_type,otodom_kitchen_type,no_kitchen_type
7182,none,separate kitchen,NaN
6360,otwarta,none,aneks kuchenny z oknem w zabudowie
6108,none,none,z oknem
5541,none,none,z oknem
1973,oddzielna,none,none
5009,none,none,oddzielna


#####  material -> DROP COLUMN

- Morizon: 66% of NONE, 66 cases
- Otodom:  50% of NONE
- NO: False values

In [228]:
display_sample_with_columns_list(sorted_df_with_prefix, ['material'])


UNIQUE VALUES FOR 'morizon_material': 51
Percentage of 'none' values for 'morizon_material': 66.65%
UNIQUE VALUES FOR 'otodom_material': 11
Percentage of 'none' values for 'otodom_material': 50.33%
UNIQUE VALUES FOR 'no_material': 5
Percentage of 'none' values for 'no_material': 0.00%
Unique values in 'no_material': ['false' 'tradycyjna cegła' 'nowa technologia' 'wielka płyta' nan]


,morizon_material,otodom_material,no_material
1231,none,none,false
1965,konstrukcja zelbetowa,concrete plate,false
6367,none,brick,tradycyjna cegła
479,mieszany,none,false
4583,rama h,none,false
1554,none,none,false


##### property_type -> DROP COLUMN
all_platforms == mieszkanie

In [229]:
display_sample_with_columns_list(sorted_df_with_prefix, ['property_type'])

UNIQUE VALUES FOR 'morizon_property_type': 1
Percentage of 'none' values for 'morizon_property_type': 0.00%
Unique values in 'morizon_property_type': ['mieszkanie']
UNIQUE VALUES FOR 'otodom_property_type': 1
Percentage of 'none' values for 'otodom_property_type': 0.00%
Unique values in 'otodom_property_type': ['mieszkanie']
UNIQUE VALUES FOR 'no_property_type': 2
Percentage of 'none' values for 'no_property_type': 0.00%
Unique values in 'no_property_type': ['mieszkanie' nan]


,morizon_property_type,otodom_property_type,no_property_type
3374,mieszkanie,mieszkanie,mieszkanie
2598,mieszkanie,mieszkanie,mieszkanie
676,mieszkanie,mieszkanie,mieszkanie
867,mieszkanie,mieszkanie,mieszkanie
2329,mieszkanie,mieszkanie,mieszkanie
6808,mieszkanie,mieszkanie,mieszkanie


In [230]:
#DROP COLUMNS
for data_frame in portal_data_objects:
    data_frame.delete_columns(['kitchen_type', 'material', 'property_type'])

### Process similar column by tematical group

In [231]:
def add_unmatched_columns_to_dict(existing_columns, **dataframes):
    columns_dict = {}
    
    for platform, df in dataframes.items():
        columns_dict[platform] = []
        for col in df.columns:
            if col not in existing_columns:
                columns_dict[platform].append(col)

    return columns_dict

unmatched_columns_names = add_unmatched_columns_to_dict(similar_column_names, morizon=morizon.df, otodom=otodom.df, no=no.df)


In [232]:
for platform, columns in unmatched_columns_names.items():
    filtered_columns = [col for col in columns if '_nan' not in col]
    print(f"{platform}: {sorted(filtered_columns)}")

morizon: ['balcony', 'bath_with_wc', 'building_type', 'building_year', 'city', 'description', 'electricity', 'gas', 'id', 'market_type', 'platform', 'prec_adress', 'primery_market', 'row_price', 'transaction_type', 'water']
otodom: ['balcony/loggia', 'building_type', 'building_year', 'city', 'date_added', 'date_update', 'id', 'id_realestateagency', 'market_type', 'ownership_form', 'portal_name', 'price', 'property_rent', 'security', 'street', 'windows']
no: ['balcony/loggia', 'bulding_type', 'bulding_year', 'date_added', 'date_update', 'electricity', 'gas', 'height', 'id_portal', 'id_realestateagency', 'makret_type', 'ownership_form', 'portal_name', 'prec_adress', 'price', 'property_rent', 'saperate_wc', 'security', 'standard', 'street', 'town/city', 'water', 'windows']


### Raname columns:
- Morizon:
    - platform: portal_name
    - row_price: price
    - prec_adress': 'street' 
- Otodom:
    - balcony/loggia: balcony
- NO:
    - balcony/loggia: balcony
    - bulding_type: building_type
    - bulding_year: building_year
    - town/city: city
    - id_portal: id
    - makret_type: market_type

In [233]:
rename_dict = {
    'morizon': {
        'platform': 'portal_name',
        'row_price': 'price',
        'prec_adress': 'street' 
    },
    'otodom': {
        'balcony/loggia': 'balcony',
    },
    'no': {
        'balcony/loggia': 'balcony',
        'bulding_type': 'building_type',
        'bulding_year': 'building_year',
        'town/city': 'city',
        'id_portal': 'id',
        'makret_type': 'market_type',
    }
}

def rename_columns(platform, df, rename_dict):
    if platform in rename_dict:
        df.rename(columns=rename_dict[platform], inplace=True)
    return df

rename_columns('morizon', morizon.df, rename_dict)
rename_columns('otodom', otodom.df, rename_dict)
rename_columns('no', no.df, rename_dict)

,balcony,basement,building_type,building_year,date_added,date_update,district,electricity,elevator,gas,...,url,water,windows,is_parking,parking_nan,property_level_nan,total_property_level_nan,elevator_nan,basement_nan,heating_type_nan
0,True,1,blok mieszkalny,1980,none,17.03.2024,ursynow,True,0,none,...,https://warszawa.nieruchomosci-online.pl/miesz...,True,plastikowe,1,0,0,0,0,0,0
1,True,0,blok mieszkalny,2000,none,11.03.2024,mokotow,True,1,none,...,https://warszawa.nieruchomosci-online.pl/miesz...,True,none,1,0,0,0,0,0,1
2,False,1,kamienica,1911,none,18.03.2024,srodmiescie,True,0,none,...,https://warszawa.nieruchomosci-online.pl/miesz...,True,plastikowe,0,1,0,0,0,0,0
3,True,0,apartamentowiec,2023,none,12.03.2024,mokotow,True,1,none,...,https://warszawa.nieruchomosci-online.pl/miesz...,True,none,0,1,0,0,0,0,0
4,True,0,"rok budowy: 2018, winda",2018,none,17.03.2024,wilanow,True,1,none,...,https://warszawa.nieruchomosci-online.pl/miesz...,True,none,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7036,False,1,blok mieszkalny,1950,none,17.03.2024,srodmiescie,True,0,none,...,https://warszawa.nieruchomosci-online.pl/miesz...,True,plastikowe,1,0,0,0,0,0,1
7037,False,0,apartamentowiec,2019,none,04.03.2024,ursynow,True,1,none,...,https://warszawa.nieruchomosci-online.pl/miesz...,True,plastikowe,1,0,0,0,0,0,1
7038,True,1,blok mieszkalny,2008,none,23.02.2024,praga południe,True,0,none,...,https://warszawa.nieruchomosci-online.pl/miesz...,True,plastikowe,1,0,0,0,0,0,0
7040,True,0,blok mieszkalny,2017,none,28.02.2024,białołeka,True,1,none,...,https://warszawa.nieruchomosci-online.pl/miesz...,True,plastikowe,1,0,0,0,0,0,0


### Drop columns (not match in all columns even in values specifications)


In [234]:
non_match_drop_columns = {
'morizon': [
    'description', 
    'bath_with_wc',
    "electricity", 
    "gas",
    "primery_market",
    "transaction_type",
    "water",
    ],
'otodom': [
    "date_added", 
    "date_update", 
    "id_realestateagency",
    "ownership_form",
    'property_rent',
    "security",
    'windows',
    ],
'no': [
    "date_added", 
    "date_update", 
    "electricity", 
    "gas", 
    "id_realestateagency",
    "ownership_form",
    "security",
    'water', 
    'windows',
    'saperate_wc', 
    'standard', 
    'height',
    'property_rent'
    ]
}

# Creating a new DataFrame that combines columns from different platforms with prefixes
# Sorting columns alphabetically without considering the platform prefix
sorted_columns = sorted(
    (column for platform, columns in non_match_drop_columns.items() for column in columns),
    key=lambda x: x
)

combined_columns_df = pd.DataFrame({
    platform + '_' + column: objects_dataframes[platform][column]
    for platform, columns in non_match_drop_columns.items()
    for column in sorted_columns if column in columns and column in objects_dataframes[platform].columns
})

combined_columns_df


,morizon_bath_with_wc,morizon_description,morizon_electricity,morizon_gas,morizon_primery_market,morizon_transaction_type,morizon_water,otodom_date_added,otodom_date_update,otodom_id_realestateagency,...,no_gas,no_height,no_id_realestateagency,no_ownership_form,no_property_rent,no_saperate_wc,no_security,no_standard,no_water,no_windows
0,0.0,Zapraszamy do zapoznania się z naszą ofertą sp...,0.0,0.0,1 540 000,sprzedaz,none,2024-03-11T11:52:35+01:00,2024-03-15T19:32:12+01:00,5077520.0,...,none,none,284958/3376/OMS,własność,1,2.0,"domofon/wideofon, monitoring/kamery",do remontu,True,plastikowe
1,0.0,Mam przyjemność zaprezentować Państwu wyjątkow...,0.0,0.0,1 560 000,sprzedaz,none,2024-03-11T12:41:55+01:00,2024-03-15T19:31:14+01:00,5077520.0,...,none,none,285188/3376/OMS,własność,1,1.0,"domofon/wideofon, monitoring/kamery, ochrona",wysoki standard,True,none
2,0.0,Mam przyjemność zaprezentować Państwu wyjątkow...,0.0,0.0,1 560 000,sprzedaz,none,2024-03-11T16:43:20+01:00,2024-03-15T19:29:27+01:00,5077520.0,...,none,none,285124/3376/OMS,własność,900,1.0,"domofon/wideofon, monitoring/kamery, ochrona",dobry,True,plastikowe
3,0.0,WE SPEAK ENGLISH – PLEASE DO NOT HESITATE TO C...,1.0,0.0,1 650 000,sprzedaz,Woda,2024-03-13T11:35:07+01:00,2024-03-15T19:28:53+01:00,5077520.0,...,none,none,2422/464/OMS,własność,none,1.0,"monitoring/kamery, ochrona",wysoki standard,True,none
4,0.0,1.Włącz lokalizator! Znajdź swoje wymarzone mi...,0.0,0.0,1 660 000,sprzedaz,none,2024-03-13T13:35:14+01:00,2024-03-15T19:26:48+01:00,5077520.0,...,none,none,1201419,"własność, księga wieczysta",1,none,none,bardzo dobry,True,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-09-04T17:42:16+02:00,2024-03-15T20:10:00+01:00,none,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-19T13:19:42+01:00,2024-03-05T13:24:47+01:00,none,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11339,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-13T15:54:47+01:00,2024-03-04T15:58:39+01:00,none,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-25T08:13:45+01:00,2024-02-25T09:33:03+01:00,none,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### morizon.primery_market looks as if it contains the price of the property, and some of the rows have been removed - however, this was checked beforehand. 
The molivities of relating columns between rows have not been identified, they will all be removed

In [235]:
print('NONE / NaN count divided by lenght and percentege share in column')
for column in combined_columns_df.columns:
    none_count = combined_columns_df[column].isna().sum() + (combined_columns_df[column] == 'NONE').sum()
    total_length = len(combined_columns_df)
    print(f"{column}:\n {none_count}/{total_length} \t{(none_count/total_length)*100:.2f}%")

NONE / NaN count divided by lenght and percentege share in column
morizon_bath_with_wc:
 5928/10731 	55.24%
morizon_description:
 5928/10731 	55.24%
morizon_electricity:
 5928/10731 	55.24%
morizon_gas:
 5928/10731 	55.24%
morizon_primery_market:
 5928/10731 	55.24%
morizon_transaction_type:
 5928/10731 	55.24%
morizon_water:
 5928/10731 	55.24%
otodom_date_added:
 529/10731 	4.93%
otodom_date_update:
 529/10731 	4.93%
otodom_id_realestateagency:
 529/10731 	4.93%
otodom_ownership_form:
 529/10731 	4.93%
otodom_property_rent:
 529/10731 	4.93%
otodom_security:
 529/10731 	4.93%
otodom_windows:
 529/10731 	4.93%
no_date_added:
 3855/10731 	35.92%
no_date_update:
 3855/10731 	35.92%
no_electricity:
 3855/10731 	35.92%
no_gas:
 3855/10731 	35.92%
no_height:
 3855/10731 	35.92%
no_id_realestateagency:
 3855/10731 	35.92%
no_ownership_form:
 3855/10731 	35.92%
no_property_rent:
 3855/10731 	35.92%
no_saperate_wc:
 3855/10731 	35.92%
no_security:
 3855/10731 	35.92%
no_standard:
 3855/10731 

In [236]:
try:
    morizon.df.drop(columns=non_match_drop_columns['morizon'], inplace=True)
except KeyError as e:
    print(f"Failed to remove columns in morizon.df: {e}")

try:
    otodom.df.drop(columns=non_match_drop_columns['otodom'], inplace=True)
except KeyError as e:
    print(f"Failed to remove columns in otodom.df: {e}")

try:
    no.df.drop(columns=non_match_drop_columns['no'], inplace=True)
except KeyError as e:
    print(f"Failed to remove columns in no.df: {e}")

### Renemed columns processing

In [237]:
similar_column_after_renamed = find_common_columns(morizon.df, otodom.df, no.df)

sorted_df_after_renamed = create_similar_dataframe(similar_column_after_renamed, morizon=morizon.df, otodom=otodom.df, no=no.df)

renamed_columns_without_similar = set(similar_column_after_renamed) - set(similar_column_names)
renamed_columns_without_similar = list(renamed_columns_without_similar)
renamed_columns_without_similar


['building_year',
 'building_type',
 'elevator_nan',
 'price',
 'total_property_level_nan',
 'market_type',
 'balcony',
 'city',
 'heating_type_nan',
 'id',
 'property_level_nan',
 'portal_name',
 'street',
 'basement_nan']

In [238]:
#PClean columns ones again
similar_columns_to_celean_text = [
 'balcony',
 'building_type',
 'building_year',
 'city',
 'market_type',
 'portal_name',
 'price',
 'street',
]

for portal_data_object in portal_data_objects:
    portal_data_object.clean_text_data(similar_columns_to_celean_text)
    if portal_data_object.df[similar_columns_to_celean_text].isnull().any().any():
        print("There are NaN values in the data - replaced with 'NONE")
        portal_data_object.complete_none()
    

In [239]:
renamed_dict = {
    #numerrical
    'price', #predicted value
    'building_year',

    #combine with district from previous section
    'street',

    'market_type',
    'building_type',
    'city',

    'balcony',
 
    'id',
    'portal_name',
    
    #nan flags
    'basement_nan',
    'elevator_nan',
    'heating_type_nan',
    'no_rooms_nan',
    'property_level_nan',
    'total_property_level_nan'
    }

In [240]:
sorted_df_with_prefix_renamed = create_similar_dataframe(similar_column_after_renamed, morizon=morizon.df, otodom=otodom.df, no=no.df)

### Compare renamed columns

##### market_type

In [241]:
display_sample_with_columns_list(sorted_df_with_prefix_renamed, ['market_type'])


UNIQUE VALUES FOR 'morizon_market_type': 2
Percentage of 'none' values for 'morizon_market_type': 0.00%
Unique values in 'morizon_market_type': ['wtorny' 'pierwotny']
UNIQUE VALUES FOR 'otodom_market_type': 3
Percentage of 'none' values for 'otodom_market_type': 0.00%
Unique values in 'otodom_market_type': ['secondary' 'primary' nan]
UNIQUE VALUES FOR 'no_market_type': 4
Percentage of 'none' values for 'no_market_type': 9.60%
Unique values in 'no_market_type': ['wtorny' 'none' nan 'pierwotny']


,morizon_market_type,otodom_market_type,no_market_type
3620,wtorny,secondary,NaN
5089,wtorny,secondary,wtorny
6040,wtorny,secondary,wtorny
1374,pierwotny,secondary,wtorny
4073,wtorny,secondary,wtorny
5439,wtorny,primary,wtorny


In [242]:
# Displaying unique values for the 'morizon_market_type' column
morizon_unique_values = sorted_df_with_prefix_renamed['morizon_market_type'].unique()
print("Unique values for morizon_market_type:", morizon_unique_values)

# Displaying unique values for the 'otodom_market_type' column
otodom_unique_values = sorted_df_with_prefix_renamed['otodom_market_type'].unique()
print("Unique values for otodom_market_type:", otodom_unique_values)

# Displaying unique values for the 'no_market_type' column
no_unique_values = sorted_df_with_prefix_renamed['no_market_type'].unique()
print("Unique values for no_market_type:", no_unique_values)


Unique values for morizon_market_type: ['wtorny' 'pierwotny']
Unique values for otodom_market_type: ['secondary' 'primary' nan]
Unique values for no_market_type: ['wtorny' 'none' nan 'pierwotny']


In [243]:
rename_market_type_dict = {
    'pierwotny': 1,
    'primary': 1,
    'wtorny': 0,
    'wtórny': 0,
    'secondary': 0,
    'none':0
}

def change_column_values_with_dict(df, col_name, rename_dict, new_col_name=None):
    if col_name in df.columns:
        df[col_name] = df[col_name].map(rename_dict).fillna(df[col_name])
        if new_col_name:
            df.rename(columns={col_name: new_col_name}, inplace=True)
    else:
        print(f"Column '{col_name}' does not exist in DataFrame.")
    return df

for data_frame in portal_data_objects:
    data_frame.create_nan_column(['market_type'])
    data_frame.df.rename(columns={'market_type_nan': 'is_primary_nan'}, inplace=True)
    change_column_values_with_dict(data_frame.df, 'market_type', rename_market_type_dict, 'is_primary')
    data_frame.unique_values(['is_primary', 'is_primary_nan'])


UNIQUE VALUES FOR 'is_primary': 2
Percentage of 'none' values for 'is_primary': 0.00%
Unique values in 'is_primary': [0 1]
UNIQUE VALUES FOR 'is_primary_nan': 1
Percentage of 'none' values for 'is_primary_nan': 0.00%
Unique values in 'is_primary_nan': [0]
UNIQUE VALUES FOR 'is_primary': 2
Percentage of 'none' values for 'is_primary': 0.00%
Unique values in 'is_primary': [0 1]
UNIQUE VALUES FOR 'is_primary_nan': 1
Percentage of 'none' values for 'is_primary_nan': 0.00%
Unique values in 'is_primary_nan': [0]
UNIQUE VALUES FOR 'is_primary': 2
Percentage of 'none' values for 'is_primary': 0.00%
Unique values in 'is_primary': [0 1]
UNIQUE VALUES FOR 'is_primary_nan': 2
Percentage of 'none' values for 'is_primary_nan': 0.00%
Unique values in 'is_primary_nan': [0 1]


#### PRICE 

In [244]:
display_sample_with_columns_list(sorted_df_with_prefix_renamed, ['price'])

UNIQUE VALUES FOR 'morizon_price': 1371
Percentage of 'none' values for 'morizon_price': 0.00%
UNIQUE VALUES FOR 'otodom_price': 1507
Percentage of 'none' values for 'otodom_price': 0.00%
UNIQUE VALUES FOR 'no_price': 1429
Percentage of 'none' values for 'no_price': 0.00%
UNIQUE VALUES FOR 'morizon_row_price_m2': 2682
Percentage of 'none' values for 'morizon_row_price_m2': 0.00%
UNIQUE VALUES FOR 'otodom_row_price_m2': 3020
Percentage of 'none' values for 'otodom_row_price_m2': 0.00%
UNIQUE VALUES FOR 'no_row_price_m2': 3026
Percentage of 'none' values for 'no_row_price_m2': 0.00%


,morizon_price,otodom_price,no_price,morizon_row_price_m2,otodom_row_price_m2,no_row_price_m2
2589,1 770 000,925000.0,750000.0,22868,18500.0,16505.0
6321,402 868,729000.0,1970000.0,15100,12972.0,30781.0
4615,2 800 000,1200000.0,1199000.0,31059,17751.0,16813.0
6658,649 000,799000.0,NaN,18543,22507.0,NaN
1952,1 450 000,725000.0,719000.0,22586,14646.0,12793.0
3556,1 020 000,1450000.0,680000.0,17895,34150.0,14782.0


In [245]:
otodom.df[(otodom.df['price'] == 'none')][['price', 'is_primary']]

,price,is_primary


In [246]:
for data_frame in portal_data_objects:
    data_frame.df['price'] = data_frame.df['price'].astype(str).str.replace(' ', '').astype(float)
    print(data_frame.df.price.unique())

[1540000. 1560000. 1650000. ... 1236922. 1342814. 1361806.]
[1290000. 1260000. 1283710. ... 5049000. 1479400. 9999999.]
[1075000. 2110000. 1890000. ... 1062000.  549999.  678999.]


### building_year -> int()

In [247]:
display_sample_with_columns_list(sorted_df_with_prefix_renamed, ['building_year'])

UNIQUE VALUES FOR 'morizon_building_year': 118
Percentage of 'none' values for 'morizon_building_year': 5.79%
UNIQUE VALUES FOR 'otodom_building_year': 133
Percentage of 'none' values for 'otodom_building_year': 7.60%
UNIQUE VALUES FOR 'no_building_year': 128
Percentage of 'none' values for 'no_building_year': 0.00%


,morizon_building_year,otodom_building_year,no_building_year
6378,2001.0,NaN,2008
4335,1954.0,NaN,2023
5264,1973.0,2019.0,2023
4104,1961.0,NaN,2003
6526,2007.0,1957.0,2023
3529,1955.0,2022.0,false


In [248]:
no.convert_bool_to_int(['building_year'])

for data_frame in portal_data_objects:

    data_frame.create_nan_column(['building_year'])
    data_frame.df['building_year'] = data_frame.df['building_year'].apply(lambda x: 0 if isinstance(x, str) and 'none' in x else x)
    data_frame.df['building_year'] = data_frame.df['building_year'].astype(int)
    
    print(f"""
    {data_frame.df['building_year'].min()}
    {data_frame.df['building_year'].median()}
    {data_frame.df['building_year'].max()}
    """)

no.df.building_year.info


    0
    2004.0
    2026
    

    0
    2008.0
    19392017
    

    0
    2003.0
    19633
    


<bound method Series.info of 0       1980
1       2000
2       1911
3       2023
4       2018
        ... 
7036    1950
7037    2019
7038    2008
7040    2017
7041       0
Name: building_year, Length: 6876, dtype: int64>

In [249]:
min_builging_year = 1890
max_building_year = 2025


for data_frame in portal_data_objects:
    print("Smallest values of building_year (from 15. index): ", sorted(data_frame.df['building_year'].unique())[0:15])
    print("Largest values of building_year (from 20. index): ", sorted(data_frame.df['building_year'].unique(), reverse=True)[0:15])
    print(f"Percentage of observations with 'building_year' < {min_builging_year}: {round((data_frame.df['building_year'] < min_builging_year).sum() / len(data_frame.df) * 100, 2)}%")
    print(f"Percentage of observations with 'building_year' > {max_building_year}: {round((data_frame.df['building_year'] > max_building_year).sum() / len(data_frame.df) * 100, 2)}%")

    data_frame.df.loc[~data_frame.df['building_year'].between(min_builging_year, max_building_year), 'building_year'] = data_frame.df['building_year'].median()

    #check
    print("Smallest values of building_year after transformation: ", sorted(data_frame.df['building_year'].unique())[0:1])
    print("Largest values of building_year after transformation: ", sorted(data_frame.df['building_year'].unique(), reverse=True)[0:1])
    print("----")

Smallest values of building_year (from 15. index):  [0, 1860, 1870, 1878, 1890, 1894, 1897, 1900, 1901, 1904, 1905, 1906, 1908, 1910, 1911]
Largest values of building_year (from 20. index):  [2026, 2025, 2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012]
Percentage of observations with 'building_year' < 1890: 5.91%
Percentage of observations with 'building_year' > 2025: 0.02%
Smallest values of building_year after transformation:  [1890]
Largest values of building_year after transformation:  [2025]
----
Smallest values of building_year (from 15. index):  [0, 1, 19, 60, 200, 210, 1505, 1822, 1830, 1836, 1838, 1860, 1869, 1870, 1875]
Largest values of building_year (from 20. index):  [19392017, 20024, 19881, 19861, 19633, 11954, 2026, 2025, 2024, 2023, 2022, 2021, 2020, 2019, 2018]
Percentage of observations with 'building_year' < 1890: 9.04%
Percentage of observations with 'building_year' > 2025: 0.65%
Smallest values of building_year after transformation:  [1

### Combine columns related to district:
- street
- prec_adress
- district

##### district
- Not always same district -> additional streets or subdistricts
- Potenialy 485 cases (NO)

In [250]:
display_sample_with_columns_list(sorted_df_with_prefix, ['district'])

UNIQUE VALUES FOR 'morizon_district': 86
Percentage of 'none' values for 'morizon_district': 0.62%
UNIQUE VALUES FOR 'otodom_district': 136
Percentage of 'none' values for 'otodom_district': 3.28%
UNIQUE VALUES FOR 'no_district': 485
Percentage of 'none' values for 'no_district': 1.89%


,morizon_district,otodom_district,no_district
1117,bemowo,bielany,mokotow
6786,mokotow,praga południe,białołeka
50,mokotow,wygledow,wilanow
1620,ursynow,imielin,słuzew
5413,praga poludnie,mokotow,praga połnoc
4995,none,mokotow,ochota


In [251]:
district_columns = ['morizon_district', 'otodom_district', 'no_district']

def preprocess_string_columns(data_frame, column_names):

    for column in column_names:
        if column in data_frame.columns:
            # Adding handling of NaN values before processing text and skipping 'NONE' values without changing their value
            modified_data = data_frame[column].apply(lambda x: unidecode.unidecode(x.lower().strip()) if isinstance(x, str) and x != 'none' else x)
            data_frame[column] = modified_data

# Performing preprocess_string_columns on sorted_df_with_prefix
preprocess_string_columns(sorted_df_with_prefix, district_columns)

In [252]:
# Function to replace subdistricts with main districts
def replace_subdistricts_with_districts(data_frame, column_name):
    # Creating a new column with replaced values based on the dictionary
    def map_districts(subdistrict):
        for district, subdistricts in districts_subdistricts_dict.items():
            if subdistrict in subdistricts:
                return district
        return subdistrict

    data_frame[column_name + '_norm'] = data_frame[column_name].apply(map_districts)

# Calling the function for the appropriate columns in the DataFrame
replace_subdistricts_with_districts(sorted_df_with_prefix, 'morizon_district')
replace_subdistricts_with_districts(sorted_df_with_prefix, 'otodom_district')
replace_subdistricts_with_districts(sorted_df_with_prefix, 'no_district')

display_sample_with_columns_list(sorted_df_with_prefix, ['district_norm'])

UNIQUE VALUES FOR 'morizon_district_norm': 84
Percentage of 'none' values for 'morizon_district_norm': 0.62%
UNIQUE VALUES FOR 'otodom_district_norm': 29
Percentage of 'none' values for 'otodom_district_norm': 3.28%
UNIQUE VALUES FOR 'no_district_norm': 358
Percentage of 'none' values for 'no_district_norm': 1.89%


,morizon_district_norm,otodom_district_norm,no_district_norm
1705,bialoleka,wawer,mokotow
1006,bemowo,wesola,mokotow
1095,ursus,wola,bialoleka
6228,bielany,bialoleka,rembertow
6447,targowek,wesola,ursynow
3701,wlochy,mokotow,mokotow


In [253]:
display_sample_with_columns_list(sorted_df_with_prefix, ['district'])

UNIQUE VALUES FOR 'morizon_district': 86
Percentage of 'none' values for 'morizon_district': 0.62%
UNIQUE VALUES FOR 'otodom_district': 136
Percentage of 'none' values for 'otodom_district': 3.28%
UNIQUE VALUES FOR 'no_district': 485
Percentage of 'none' values for 'no_district': 1.89%
UNIQUE VALUES FOR 'morizon_district_norm': 84
Percentage of 'none' values for 'morizon_district_norm': 0.62%
UNIQUE VALUES FOR 'otodom_district_norm': 29
Percentage of 'none' values for 'otodom_district_norm': 3.28%
UNIQUE VALUES FOR 'no_district_norm': 358
Percentage of 'none' values for 'no_district_norm': 1.89%


,morizon_district,otodom_district,no_district,morizon_district_norm,otodom_district_norm,no_district_norm
2412,ursus,bielany,srodmiescie,ursus,bielany,srodmiescie
1695,praga polnoc,rembertow,bemowo,praga polnoc,rembertow,bemowo
4334,mokotow,goclaw,mokotow,mokotow,praga poludnie,mokotow
3663,bielany,mokotow,ursynow,bielany,mokotow,ursynow
5846,wawer,srodmiescie,bielany,wawer,srodmiescie,bielany
649,wlochy,mokotow,tarchomin,wlochy,mokotow,bialoleka


In [254]:
# Counting occurrences of values in columns *_district_norm that are not in the dictionary
columns_to_check = ['morizon_district_norm', 'otodom_district_norm', 'no_district_norm']
undocumented_district_counts = {}

# Getting keys from the dictionary
dict_keys = set(districts_subdistricts_dict.keys())

for column in columns_to_check:
    for value in sorted_df_with_prefix[column]:
        if value not in dict_keys:
            if value in undocumented_district_counts:
                undocumented_district_counts[value] += 1
            else:
                undocumented_district_counts[value] = 1

# Displaying results
print("Number of occurrences of values not in the dictionary:")
print(sorted(undocumented_district_counts.items(), key=lambda item: item[1], reverse=True))




Number of occurrences of values not in the dictionary:
[('none', 423), (nan, 266), ('praga', 69), ('zakroczym', 23), ('chelmzynska', 9), ('marymont', 9), ('koszyki', 9), ('ostrobramska', 8), ('potok', 8), ('aleja komisji edukacji narodowej', 7), ('ludwiki', 6), ('zielona', 6), ('wiktoryn', 6), ('aleja wilanowska', 6), ('kapucka', 6), ('domaniewska', 5), ('pulawska', 5), ('odynca', 5), ('kolejowa', 5), ('optykow', 5), ('gorczewska', 4), ('bagno', 4), ('bukowinska', 4), ('grzybowska', 4), ('szeligowska', 4), ('sewastopolska', 4), ('korotynskiego', 4), ('gornoslaska', 4), ('nowodworska', 4), ('harfowa', 4), ('wilhelma', 3), ('guminska', 3), ('fasolowa', 3), ('piekna', 3), ('malborska', 3), ('zlota', 3), ('sulejkowska', 3), ('leona', 3), ('konstruktorska', 3), ('czerniakowska', 3), ('lipska', 3), ('szczesliwicka', 3), ('wlodarzewska', 3), ('grochowska', 3), ('sielecka', 3), ('talarowa', 3), ('dickensa', 3), ('paryska', 3), ('pory', 3), ('aleja jerzego waszyngtona', 3), ('aleja niepodleglos

##### Most of the remaining values are streets (including with addresses and in various forms, e.g., solidarnosci vs. solidarnosci avenue). Additionally, identified were:
'stara iwiczna' - outside of warsaw
'praga' - a problem with assigning the appropriate district

Insofar as the results were to be improved in the future, it is molive to try and overwrite the streets with the names of the districts while normalizing them.

In [255]:
# Calculating the percentage of values that are not keys in the districts_subdistricts_dict
undocumented_district_percentages = {}
total_rows = len(sorted_df_with_prefix)
total_undocumented = 0
for column in columns_to_check:
    undocumented_count = sorted_df_with_prefix[column][~sorted_df_with_prefix[column].isin(dict_keys)].count()
    undocumented_district_percentages[column] = (undocumented_count / total_rows) * 100
    total_undocumented += undocumented_count

overall_percentage = (total_undocumented / (total_rows * len(columns_to_check))) * 100

print("Percentage of values not in the dictionary for each column:")
print({k: round(v, 2) for k, v in undocumented_district_percentages.items()})
print("Overall percentage of values not in the dictionary in all columns:", round(overall_percentage, 2), '%')


Percentage of values not in the dictionary for each column:
{'morizon_district_norm': 2.18, 'otodom_district_norm': 4.5, 'no_district_norm': 9.3}
Overall percentage of values not in the dictionary in all columns: 5.33 %


In [256]:
#Chcnge rest of values (5,5% of all rows) to uknnown
for data_frame in portal_data_objects:
    preprocess_string_columns(data_frame.df, ['district'])
    replace_subdistricts_with_districts(data_frame.df, 'district')
    data_frame.df['district_norm'] = data_frame.df['district_norm'].apply(lambda x: 'UNKNOWN' if x not in districts_subdistricts_dict.keys() else x)
    data_frame.unique_values(['district_norm'])
    


UNIQUE VALUES FOR 'district_norm': 19
Percentage of 'none' values for 'district_norm': 0.00%
UNIQUE VALUES FOR 'district_norm': 19
Percentage of 'none' values for 'district_norm': 0.00%
UNIQUE VALUES FOR 'district_norm': 19
Percentage of 'none' values for 'district_norm': 0.00%


In [257]:
def create_dataframe_with_separeted_col(columns_list, **portal_data_objects):
    # Creating an empty DataFrame
    extracterd_columns_df = pd.DataFrame()
    
    # Iterating through each portal data object
    for portal_name, portal in portal_data_objects.items():
        # Checking which columns from the list are available in the DataFrame
        available_columns = [col for col in columns_list if col in portal.columns]
        # Creating a temporary DataFrame with available columns and adding a prefix
        temp_df = portal[available_columns].copy()
        temp_df.columns = [f"{portal_name}_{col}" for col in temp_df.columns]
        
        # Merging the DataFrame with the main DataFrame
        if extracterd_columns_df.empty:
            extracterd_columns_df = temp_df
        else:
            extracterd_columns_df = pd.merge(extracterd_columns_df, temp_df, how='outer', left_index=True, right_index=True)
    
    return extracterd_columns_df

localization_columns = ['district_norm', 'street', 'prec_adress']
seperated_localization_df = create_dataframe_with_separeted_col(columns_list=localization_columns, morizon=morizon.df, otodom=otodom.df, no=no.df)

In [258]:
display_sample_with_columns_list(seperated_localization_df, localization_columns)

UNIQUE VALUES FOR 'morizon_district_norm': 20
Percentage of 'none' values for 'morizon_district_norm': 0.00%
UNIQUE VALUES FOR 'morizon_street': 1249
Percentage of 'none' values for 'morizon_street': 6.50%
UNIQUE VALUES FOR 'otodom_district_norm': 20
Percentage of 'none' values for 'otodom_district_norm': 0.00%
UNIQUE VALUES FOR 'otodom_street': 1536
Percentage of 'none' values for 'otodom_street': 24.97%
UNIQUE VALUES FOR 'no_district_norm': 20
Percentage of 'none' values for 'no_district_norm': 0.00%
UNIQUE VALUES FOR 'no_street': 1353
Percentage of 'none' values for 'no_street': 13.95%
UNIQUE VALUES FOR 'no_prec_adress': 185
Percentage of 'none' values for 'no_prec_adress': 61.32%


,morizon_district_norm,morizon_street,otodom_district_norm,otodom_street,no_district_norm,no_street,no_prec_adress
7840,NaN,NaN,ochota,ul łopuszanska,NaN,NaN,NaN
4861,wilanow,adama branickiego,praga poludnie,ul witolinska,mokotow,wołoska,none
3897,zoliborz,ludwika rydygiera,bielany,ul zbigniewa romaszewskiego,srodmiescie,piwna,none
4634,NaN,NaN,wola,ul ludwiki,UNKNOWN,none,none
8634,NaN,NaN,mokotow,ul stefana pienkowskiego,NaN,NaN,NaN
2801,mokotow,jana iii sobieskiego,ursus,ul posag 7 panien,wawer,czecha,none


##### street and prec_adres (NO)

##### street columns are non-standardized in many respects, and are also very numerous (about 1,500 per portal). As mentioned earlier, they should be standardized in the future based on the street dictionary. For the moment, they will be removed.

prec_address in NO >60% NONE -> drop

in connection with the normalization of the district in another column, it will be removed

In [259]:
#drop prec_adress and streets
localization_columns_to_drop = ['district', 'street', 'prec_adress']

for column in localization_columns_to_drop:
    for data_frame in portal_data_objects:
        if column in data_frame.df.columns:
            data_frame.delete_columns(column)

'district' in morizon.df.columns

False

#### building_type
- morizon: 68% NONE values
- no: if 'rok budowy' in  bulding_type (rok budowy: 1986) -> delete r'rok budowy: ****
- no: delete winda 
-all platforms: unify dict to volues ['tenement', 'apartment', 'NONE', 'block', 'ribbon', 'house', 'other']

In [260]:
display_sample_with_columns_list(sorted_df_with_prefix_renamed, ['building_type'])

UNIQUE VALUES FOR 'morizon_building_type': 10
Percentage of 'none' values for 'morizon_building_type': 52.72%
UNIQUE VALUES FOR 'otodom_building_type': 8
Percentage of 'none' values for 'otodom_building_type': 15.30%
UNIQUE VALUES FOR 'no_building_type': 364
Percentage of 'none' values for 'no_building_type': 3.39%


,morizon_building_type,otodom_building_type,no_building_type
5019,blok,tenement,blok mieszkalny
6643,apartamentowiec,NaN,kamienica
6501,kamienica,NaN,blok mieszkalny
3301,apartamentowiec,block,rok budowy 2006 blok w nowym budownictwie winda
5516,kamienica,apartment,rok budowy 2022 winda
479,apartamentowiec,none,blok mieszkalny


In [261]:
otodom_uniqe_btype = otodom.df['building_type'].unique()
otodom_uniqe_btype

array(['block', 'apartment', 'tenement', 'none', 'ribbon', 'house',
       'infill', 'loft'], dtype=object)

##### NO building_type

In [262]:
def extract_building_technic(building_type):
    if isinstance(building_type, str):
        technic = re.search(r'technika budowy: ([^,]+)', building_type)
        return technic.group(1) if technic else None
    return None

no_building_df = no.df['building_type'].astype(str).str.replace('winda', '', regex=False)
no_building_df = no_building_df.str.replace(r'rok budowy: \d{4}', '', regex=True)
no_building_df['building_technic'] = no_building_df.apply(extract_building_technic)
no_building_df = no_building_df.str.replace(r'technika budowy: [^,]+,?', '', regex=True)
no_building_df = no_building_df.apply(lambda x: 'block' if isinstance(x, str) and 'blok' in x else x)

print(no_building_df.unique())

['block' 'kamienica' 'apartamentowiec' 'rok budowy 2018 '
 'rok budowy 2000 ' 'rok budowy 2001 ' 'none' 'rok budowy 1973 '
 'rok budowy 2024' 'rok budowy 2005' 'rok budowy 2023' 'dom wielorodzinny'
 'rok budowy 2012 ' 'rok budowy 2024 ' 'rok budowy 2008' 'rok budowy 1928'
 'rok budowy 1978 ' 'rok budowy 1994 ' 'rok budowy 2005 '
 'dom wolnostojacy' 'rok budowy 1975'
 'rok budowy 1980 technika budowy wielka płyta ' 'rok budowy 2000'
 'rok budowy 2008 technika budowy tradycyjna cegła '
 'rok budowy 2020 technika budowy tradycyjna cegła ' 'rok budowy 1962'
 'rok budowy 2004' 'rok budowy 1998 technika budowy tradycyjna cegła'
 'rok budowy 2002 ' 'rok budowy 1996 ' 'rok budowy 1976'
 'rok budowy 1988 ' 'rok budowy 1995' 'rok budowy 2004 ' 'rok budowy 2021'
 'rok budowy 2006 ' 'rok budowy 1982' 'rok budowy 2023 ' 'rok budowy 1998'
 'rok budowy 1969 technika budowy tradycyjna cegła'
 'rok budowy 2012 technika budowy tradycyjna cegła'
 'apartamentowiec  dostosowany do osob niepełnosprawnych'
 

In [263]:
no_building_rename_dict = {
    'kamienica': 'tenement', 
    'apartamentowiec': 'apartment',  
    'apartament': 'apartment', 
    'dom wielorodzinny': 'tenement', 
    'dom wolnostojący': 'house', 
    'dom': 'house',
    'rezydencja': 'house',  
    'loft': 'loft',
}

def rename_building_type(building_type):
    if isinstance(building_type, str):
        parts = building_type.split(',')
        for part in parts:
            part = part.strip()
            if part in no_building_rename_dict:
                return no_building_rename_dict[part]
    return 'none'

no_building_df = no.df['building_type'].astype(str).apply(rename_building_type)
no.df['building_type'] = no_building_df

no.df['building_type'].unique()

array(['none', 'tenement', 'apartment', 'house'], dtype=object)

##### Morizon building_type

In [264]:
morizon.df['building_type'].unique()


array(['kamienica', 'apartamentowiec', 'none', 'blok', 'segment', 'dom',
       'szeregowy', 'wielorodzinny', 'inna', 'inny'], dtype=object)

In [265]:
morizon_building_rename_dict = {
    'apartamentowiec': 'apartment',
    'blok': 'block',
    'dom': 'house',
    'inna': 'other',
    'inny': 'other',
    'kamienica': 'tenement',
    'segment': 'ribbon',
    'szeregowy': 'ribbon',
    'wielorodzinny': 'house'
}

morizon.df['building_type'] = morizon.df['building_type'].map(morizon_building_rename_dict).fillna(morizon.df['building_type'])


In [266]:
morizon.df['building_type'].unique()

array(['tenement', 'apartment', 'none', 'block', 'ribbon', 'house',
       'other'], dtype=object)

In [267]:
#refresh silimiar dataframe
sorted_df_after_rename = create_similar_dataframe(similar_column_after_renamed, morizon=morizon.df, otodom=otodom.df, no=no.df)


In [268]:
display_sample_with_columns_list(sorted_df_after_rename, ['building_type'])

UNIQUE VALUES FOR 'morizon_building_type': 7
Percentage of 'none' values for 'morizon_building_type': 52.72%
UNIQUE VALUES FOR 'otodom_building_type': 8
Percentage of 'none' values for 'otodom_building_type': 15.30%
UNIQUE VALUES FOR 'no_building_type': 5
Percentage of 'none' values for 'no_building_type': 62.32%
Unique values in 'no_building_type': ['none' 'tenement' 'apartment' nan 'house']


,morizon_building_type,otodom_building_type,no_building_type
6784,apartment,apartment,apartment
5810,none,apartment,tenement
5146,none,block,none
3829,none,NaN,none
136,block,apartment,none
5779,none,NaN,apartment


In [269]:
for data_frame in portal_data_objects:
    data_frame.create_nan_column(['building_type'])
    data_frame.df['building_type'] = data_frame.df['building_type'].apply(lambda x: 'UNKNOWN' if isinstance(x, str) and (x == 'none' or x == 'other') else x)
    data_frame.unique_values(['building_type'])


UNIQUE VALUES FOR 'building_type': 6
Percentage of 'none' values for 'building_type': 0.00%
UNIQUE VALUES FOR 'building_type': 8
Percentage of 'none' values for 'building_type': 0.00%
UNIQUE VALUES FOR 'building_type': 4
Percentage of 'none' values for 'building_type': 0.00%
Unique values in 'building_type': ['UNKNOWN' 'tenement' 'apartment' 'house']


#### city

In [270]:
city_column = ['morizon_city', 'otodom_city', 'no_city']

preprocess_string_columns(sorted_df_after_rename, city_column)

display_sample_with_columns_list(sorted_df_after_rename, ['city'])

UNIQUE VALUES FOR 'morizon_city': 2
Percentage of 'none' values for 'morizon_city': 0.00%
Unique values in 'morizon_city': ['warszawa' 'nowodworski']
UNIQUE VALUES FOR 'otodom_city': 6
Percentage of 'none' values for 'otodom_city': 0.00%
UNIQUE VALUES FOR 'no_city': 7
Percentage of 'none' values for 'no_city': 0.00%


,morizon_city,otodom_city,no_city
122,warszawa,warszawa,warszawa
5580,warszawa,warszawa,warszawa
59,warszawa,warszawa,warszawa
6953,warszawa,warszawa,warszawa
3917,warszawa,warszawa,warszawa
4952,warszawa,warszawa,warszawa


In [271]:
for data_frame in portal_data_objects:
    data_frame.unique_values(['city'])
    print(data_frame.df['city'].unique())

UNIQUE VALUES FOR 'city': 2
Percentage of 'none' values for 'city': 0.00%
Unique values in 'city': ['warszawa' 'nowodworski']
['warszawa' 'nowodworski']
UNIQUE VALUES FOR 'city': 9
Percentage of 'none' values for 'city': 0.00%
['warszawa' 'raszyn' 'zabki' 'macierzysz' 'piastow' 'marki' 'mysiadlo'
 'bialoleka' 'dawidy bankowe']
UNIQUE VALUES FOR 'city': 6
Percentage of 'none' values for 'city': 0.00%
['warszawa' 'otrebusy' 'łomianki' 'piaseczno' 'janczewice'
 'ozarow mazowiecki']


In [272]:
otodom.df['city'] = otodom.df['city'].apply(lambda x: 'warszawa' if isinstance(x, str) and 'bialoleka' in x else x)

In [273]:
for data_frame in portal_data_objects:
    data_frame.df.drop(data_frame.df[data_frame.df['city'] != 'warszawa'].index, inplace=True)
    data_frame.unique_values(['city'])
    #only one value in column - sure we have only warsaw offers
    data_frame.delete_columns(['city'])

UNIQUE VALUES FOR 'city': 1
Percentage of 'none' values for 'city': 0.00%
Unique values in 'city': ['warszawa']
UNIQUE VALUES FOR 'city': 1
Percentage of 'none' values for 'city': 0.00%
Unique values in 'city': ['warszawa']
UNIQUE VALUES FOR 'city': 1
Percentage of 'none' values for 'city': 0.00%
Unique values in 'city': ['warszawa']


#### balcony

In [274]:
display_sample_with_columns_list(sorted_df_after_rename, ['balcony'])

UNIQUE VALUES FOR 'morizon_balcony': 2
Percentage of 'none' values for 'morizon_balcony': 0.00%
Unique values in 'morizon_balcony': [0. 1.]
UNIQUE VALUES FOR 'otodom_balcony': 3
Percentage of 'none' values for 'otodom_balcony': 4.81%
Unique values in 'otodom_balcony': [1.0 'none' nan]
UNIQUE VALUES FOR 'no_balcony': 4
Percentage of 'none' values for 'no_balcony': 21.84%
Unique values in 'no_balcony': [True False 'none' nan]


,morizon_balcony,otodom_balcony,no_balcony
2711,1.0,1.0,True
3730,1.0,1.0,False
4044,0.0,1.0,True
105,1.0,1.0,True
4282,0.0,NaN,True
1604,0.0,1.0,none


In [275]:
for data_frame in portal_data_objects:
    data_frame.create_nan_column(['balcony'])
    data_frame.replace_none_with_binary(['balcony'])
    if data_frame == no:
        data_frame.convert_bool_to_int(['balcony'])
    data_frame.unique_values(['balcony'])

UNIQUE VALUES FOR 'balcony': 2
Percentage of 'none' values for 'balcony': 0.00%
Unique values in 'balcony': [0. 1.]
UNIQUE VALUES FOR 'balcony': 2
Percentage of 'none' values for 'balcony': 0.00%
Unique values in 'balcony': [1. 0.]
UNIQUE VALUES FOR 'balcony': 2
Percentage of 'none' values for 'balcony': 0.00%
Unique values in 'balcony': [1 0]


#### id

In [276]:
display_sample_with_columns_list(sorted_df_after_rename, ['id', 'url'])

UNIQUE VALUES FOR 'morizon_id': 4501
Percentage of 'none' values for 'morizon_id': 0.00%
UNIQUE VALUES FOR 'otodom_id': 4418
Percentage of 'none' values for 'otodom_id': 0.00%
UNIQUE VALUES FOR 'no_id': 4494
Percentage of 'none' values for 'no_id': 0.00%
UNIQUE VALUES FOR 'morizon_url': 4501
Percentage of 'none' values for 'morizon_url': 0.00%
UNIQUE VALUES FOR 'otodom_url': 4418
Percentage of 'none' values for 'otodom_url': 0.00%
UNIQUE VALUES FOR 'no_url': 4494
Percentage of 'none' values for 'no_url': 0.00%


,morizon_id,otodom_id,no_id,morizon_url,otodom_url,no_url
4967,mzn2043416756,65186089.0,24767203,https://www.morizon.pl/oferta/sprzedaz-mieszka...,3-pokoje-mokotow-2024-iiiq-ID4pwSx,https://warszawa.nieruchomosci-online.pl/miesz...
6906,mzn2043480003,65131321.0,24711628,https://www.morizon.pl/oferta/sprzedaz-mieszka...,sprzedam-mieszkanie-na-atrakcyjnym-bemowie-ID4...,https://warszawa.nieruchomosci-online.pl/miesz...
85,mzn2043368944,65223780.0,24731204,https://www.morizon.pl/oferta/sprzedaz-mieszka...,kawalerka-metro-wilanowska-balkon-do-wprowadz-...,https://warszawa.nieruchomosci-online.pl/miesz...
316,mzn2043366512,65162448.0,24726651,https://www.morizon.pl/oferta/sprzedaz-mieszka...,okazja-dwa-tarasy-przestrzenne-3-pokoje-kabaty...,https://warszawa.nieruchomosci-online.pl/miesz...
4499,mzn2043229905,65191942.0,24805301,https://www.morizon.pl/oferta/sprzedaz-mieszka...,3-pokojowe-mieszkanie-w-dobrej-lokalizacji-ID4...,https://warszawa.nieruchomosci-online.pl/miesz...
1676,mzn2042729551,64743400.0,24521557,https://www.morizon.pl/oferta/sprzedaz-mieszka...,komfortowy-przestronny-jasny-apartament-ID4nEIo,https://warszawa.nieruchomosci-online.pl/miesz...


#### portal_name

In [277]:
display_sample_with_columns_list(sorted_df_after_rename, ['portal_name'])

UNIQUE VALUES FOR 'morizon_portal_name': 1
Percentage of 'none' values for 'morizon_portal_name': 0.00%
Unique values in 'morizon_portal_name': ['morizon']
UNIQUE VALUES FOR 'otodom_portal_name': 2
Percentage of 'none' values for 'otodom_portal_name': 0.00%
Unique values in 'otodom_portal_name': ['otodom' nan]
UNIQUE VALUES FOR 'no_portal_name': 2
Percentage of 'none' values for 'no_portal_name': 0.00%
Unique values in 'no_portal_name': ['nieruchomosci online' nan]


,morizon_portal_name,otodom_portal_name,no_portal_name
4611,morizon,otodom,nieruchomosci online
2248,morizon,otodom,nieruchomosci online
2446,morizon,otodom,nieruchomosci online
6882,morizon,otodom,NaN
2555,morizon,otodom,nieruchomosci online
849,morizon,otodom,nieruchomosci online


In [278]:
rename_market_type_dict = {
    'otodom':  'otodom',
    'nieruchomosci online': 'no'
}
for data_frame in portal_data_objects:
    change_column_values_with_dict(data_frame.df, 'portal_name', rename_market_type_dict)

otodom.df.portal_name.unique()

array(['otodom'], dtype=object)

## Check

In [279]:
final_column_names = find_common_columns(morizon.df, otodom.df, no.df)
sorted_final_column_names = sorted(final_column_names)
sorted_final_column_names

['balcony',
 'balcony_nan',
 'basement',
 'basement_nan',
 'building_type',
 'building_type_nan',
 'building_year',
 'building_year_nan',
 'district_norm',
 'elevator',
 'elevator_nan',
 'heating_type',
 'heating_type_nan',
 'id',
 'is_parking',
 'is_primary',
 'is_primary_nan',
 'no_rooms',
 'parking_nan',
 'portal_name',
 'price',
 'property_level',
 'property_level_nan',
 'row_price_m2',
 'total_area',
 'total_property_level',
 'total_property_level_nan',
 'url']

In [280]:
morizon.df[sorted_final_column_names]

,balcony,balcony_nan,basement,basement_nan,building_type,building_type_nan,building_year,building_year_nan,district_norm,elevator,...,parking_nan,portal_name,price,property_level,property_level_nan,row_price_m2,total_area,total_property_level,total_property_level_nan,url
0,0.0,0,1,0,tenement,0,1953,0,mokotow,1,...,0,morizon,1540000.0,4.0,0,16862,91.0,5.0,0,https://www.morizon.pl/oferta/sprzedaz-mieszka...
1,1.0,0,0,0,apartment,0,2022,0,bielany,1,...,0,morizon,1560000.0,3.0,0,22125,70.0,5.0,0,https://www.morizon.pl/oferta/sprzedaz-mieszka...
2,1.0,0,0,0,apartment,0,2022,0,ursynow,1,...,0,morizon,1560000.0,3.0,0,22125,110.0,5.0,0,https://www.morizon.pl/oferta/sprzedaz-mieszka...
3,1.0,0,0,0,apartment,0,2012,0,mokotow,1,...,0,morizon,1650000.0,5.0,0,21484,76.0,7.0,0,https://www.morizon.pl/oferta/sprzedaz-mieszka...
4,1.0,0,1,0,apartment,0,2004,0,ursynow,1,...,0,morizon,1660000.0,1.0,0,19302,86.0,5.0,0,https://www.morizon.pl/oferta/sprzedaz-mieszka...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7301,0.0,0,1,0,tenement,0,1952,0,srodmiescie,0,...,0,morizon,1395000.0,3.0,0,32299,43.0,3.0,0,https://www.morizon.pl/oferta/sprzedaz-mieszka...
7302,0.0,0,1,0,tenement,0,1952,0,wlochy,0,...,0,morizon,1395000.0,3.0,0,32299,100.0,3.0,0,https://www.morizon.pl/oferta/sprzedaz-mieszka...
7303,0.0,0,1,0,UNKNOWN,1,1990,0,ochota,0,...,0,morizon,1399000.0,1.0,0,16695,83.0,1.0,0,https://www.morizon.pl/oferta/sprzedaz-mieszka...
7304,1.0,0,0,0,apartment,0,2024,0,praga poludnie,0,...,0,morizon,1400000.0,1.0,0,17113,81.0,4.0,0,https://www.morizon.pl/oferta/sprzedaz-mieszka...


In [281]:
otodom.df[sorted_final_column_names]

,balcony,balcony_nan,basement,basement_nan,building_type,building_type_nan,building_year,building_year_nan,district_norm,elevator,...,parking_nan,portal_name,price,property_level,property_level_nan,row_price_m2,total_area,total_property_level,total_property_level_nan,url
0,1.0,0,0,1,block,0,2008,0,bemowo,0,...,0,otodom,1290000.0,1.0,0,16290,79.19,7.0,0,mieszkanie-79-19-m-warszawa-ID4pzV9
1,1.0,0,0,1,apartment,0,2017,0,targowek,1,...,0,otodom,1260000.0,4.0,0,18491,68.14,11.0,0,3-pokoje-moderna-osiedle-gotowe-do-wprowadzeni...
2,1.0,0,0,1,tenement,0,2007,0,targowek,1,...,1,otodom,1283710.0,4.0,0,15500,82.82,5.0,0,trzypokojowe-mieszkanie-na-pradze-polnoc-ID4pAwp
3,1.0,0,0,1,apartment,0,2014,0,bialoleka,1,...,0,otodom,1350000.0,2.0,0,18367,73.50,3.0,0,apartament-w-marinie-3-pokoje-ID4pCuq
4,0.0,1,0,1,tenement,0,1949,0,mokotow,0,...,1,otodom,1580000.0,3.0,0,22332,70.75,4.0,0,mieszkanie-w-sercu-starego-mokotowa-ID4pCJK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11337,1.0,0,1,0,apartment,0,2019,0,ursynow,1,...,0,otodom,1100000.0,3.0,0,16222,67.81,4.0,0,3-pok-klobucka-6a-komorka-obok-mieszkania-2gar...
11338,1.0,0,1,0,apartment,0,2010,0,mokotow,1,...,0,otodom,700000.0,3.0,0,33767,89.00,5.0,0,apartament-mokotow-chodkiewicza-ID4peUU
11339,1.0,0,1,0,block,0,2008,0,UNKNOWN,1,...,0,otodom,1000000.0,10.0,0,33333,30.00,1.0,0,x-ID4p9qQ
11340,1.0,0,0,1,block,0,2001,0,bialoleka,1,...,0,otodom,699000.0,4.0,0,15196,46.00,6.0,0,warszawa-tarchomin-2-pokoje-46m2-po-remoncie-I...


In [282]:
no.df[sorted_final_column_names]

,balcony,balcony_nan,basement,basement_nan,building_type,building_type_nan,building_year,building_year_nan,district_norm,elevator,...,parking_nan,portal_name,price,property_level,property_level_nan,row_price_m2,total_area,total_property_level,total_property_level_nan,url
0,1,0,1,0,UNKNOWN,1,1980,0,ursynow,0,...,0,no,1075000.0,1.0,0,12647,85.00,4.0,0,https://warszawa.nieruchomosci-online.pl/miesz...
1,1,0,0,0,UNKNOWN,1,2000,0,mokotow,1,...,0,no,2110000.0,2.0,0,19905,106.00,7.0,0,https://warszawa.nieruchomosci-online.pl/miesz...
2,0,0,1,0,tenement,0,1911,0,srodmiescie,0,...,1,no,1890000.0,0.0,0,34363,55.00,4.0,0,https://warszawa.nieruchomosci-online.pl/miesz...
3,1,0,0,0,apartment,0,2023,0,mokotow,1,...,1,no,1590000.0,4.0,0,24267,65.52,8.0,0,https://warszawa.nieruchomosci-online.pl/miesz...
4,1,0,0,0,UNKNOWN,1,2018,0,wilanow,1,...,0,no,1530000.0,2.0,0,21932,69.76,4.0,0,https://warszawa.nieruchomosci-online.pl/miesz...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7036,0,0,1,0,UNKNOWN,1,1950,0,srodmiescie,0,...,0,no,678999.0,2.0,0,19970,34.00,4.0,0,https://warszawa.nieruchomosci-online.pl/miesz...
7037,0,0,0,0,apartment,0,2019,0,ursynow,1,...,0,no,1090000.0,0.0,0,23226,46.93,3.0,0,https://warszawa.nieruchomosci-online.pl/miesz...
7038,1,0,1,0,UNKNOWN,1,2008,0,praga poludnie,0,...,0,no,799000.0,1.0,0,15380,51.95,2.0,0,https://warszawa.nieruchomosci-online.pl/miesz...
7040,1,0,0,0,UNKNOWN,1,2017,0,bialoleka,1,...,0,no,820000.0,3.0,0,13921,58.90,5.0,0,https://warszawa.nieruchomosci-online.pl/miesz...


In [283]:
# Collecting columns from each DataFrame

#unmatched_columns_names
columns_dict = {name: set(df.columns) for name, df in objects_dataframes.items()}

# Determining common columns
common_columns = set.intersection(*columns_dict.values())
print("Common columns for all platforms:")
for column in common_columns:
    print(f" - {column}")

# Determining unique columns
unique_columns = set.union(*columns_dict.values()) - common_columns
print("\nUnique columns (not common for all platforms):")
for column in unique_columns:
    print(f" - {column}")

# Checking common and unique columns in DataFrames
# Collecting columns from each DataFrame
columns_dict = {name: set(df.columns) for name, df in objects_dataframes.items()}

# Determining common columns
common_columns = set.intersection(*columns_dict.values())
print("Common columns for all platforms:")
for column in common_columns:
    print(f" - {column}")

# Determining unique columns
unique_columns = set.union(*columns_dict.values()) - common_columns
print("\nUnique columns (not common for all platforms):")
for column in unique_columns:
    print(f" - {column}")


Common columns for all platforms:
 - is_primary_nan
 - total_area
 - basement
 - elevator
 - balcony
 - property_level
 - heating_type_nan
 - balcony_nan
 - row_price_m2
 - total_property_level
 - parking_nan
 - url
 - building_type_nan
 - building_year
 - heating_type
 - price
 - is_primary
 - total_property_level_nan
 - elevator_nan
 - id
 - is_parking
 - no_rooms
 - district_norm
 - building_type
 - property_level_nan
 - building_year_nan
 - portal_name
 - basement_nan

Unique columns (not common for all platforms):
Common columns for all platforms:
 - is_primary_nan
 - total_area
 - basement
 - elevator
 - balcony
 - property_level
 - heating_type_nan
 - balcony_nan
 - row_price_m2
 - total_property_level
 - parking_nan
 - url
 - building_type_nan
 - building_year
 - heating_type
 - price
 - is_primary
 - total_property_level_nan
 - elevator_nan
 - id
 - is_parking
 - no_rooms
 - district_norm
 - building_type
 - property_level_nan
 - building_year_nan
 - portal_name
 - basement_na

In [284]:
to_int_columns = ['is_parking','no_rooms','property_level_nan','building_year_nan','property_level','total_property_level_nan','is_primary','building_year', 'elevator_nan','balcony','total_property_level','heating_type_nan','basement_nan','parking_nan','building_type_nan']

for data_frame in portal_data_objects:
    data_frame.convert_column_types(to_int_columns, int)
    

In [285]:
combined_df = pd.DataFrame(columns=list(common_columns))
for name, data_frame in objects_dataframes.items():

    combined_df = pd.concat([combined_df, data_frame[list(common_columns)]], ignore_index=True)


In [286]:
combined_df.to_csv('all_platforms_data.csv', index=False)

all_platforms_data = pd.read_csv('all_platforms_data.csv')
all_platforms_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21830 entries, 0 to 21829
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   is_primary_nan            21830 non-null  int64  
 1   total_area                21830 non-null  float64
 2   basement                  21830 non-null  int64  
 3   elevator                  21830 non-null  int64  
 4   balcony                   21830 non-null  int64  
 5   property_level            21830 non-null  int64  
 6   heating_type_nan          21830 non-null  int64  
 7   balcony_nan               21830 non-null  int64  
 8   row_price_m2              21830 non-null  int64  
 9   total_property_level      21830 non-null  int64  
 10  parking_nan               21830 non-null  int64  
 11  url                       21830 non-null  object 
 12  building_type_nan         21830 non-null  int64  
 13  building_year             21830 non-null  int64  
 14  heatin

In [287]:
all_platforms_data.sample(10)

,is_primary_nan,total_area,basement,elevator,balcony,property_level,heating_type_nan,balcony_nan,row_price_m2,total_property_level,...,elevator_nan,id,is_parking,no_rooms,district_norm,building_type,property_level_nan,building_year_nan,portal_name,basement_nan
533,0,41.00,0,0,1,100,1,0,15254,100,...,0,mzn2043485631,1,2,bialoleka,UNKNOWN,1,0,morizon,0
4252,0,46.00,1,1,1,6,0,0,18000,7,...,0,mzn2043417644,1,2,bemowo,block,0,0,morizon,0
12938,0,53.42,0,1,1,3,0,0,11100,5,...,0,65061435,1,3,rembertow,UNKNOWN,0,0,otodom,1
14602,0,55.00,1,1,1,0,0,0,11618,2,...,0,64572046,1,4,bialoleka,block,0,0,otodom,0
16342,0,53.97,0,1,0,0,1,1,14508,3,...,0,24374637,0,3,bialoleka,UNKNOWN,0,0,no,1
8713,0,41.00,0,1,1,6,1,0,14610,100,...,0,65103186,0,2,targowek,block,0,0,otodom,1
17299,0,65.78,0,1,1,1,0,0,28868,2,...,0,24567045,1,3,bielany,apartment,0,0,no,0
10320,0,27.00,0,1,1,100,1,0,21667,16,...,0,65019062,0,2,wola,block,0,0,otodom,1
19810,0,62.50,1,1,1,6,0,0,13600,11,...,0,24690985,0,3,bialoleka,UNKNOWN,0,0,no,0
21799,0,42.00,1,0,0,4,0,0,19999,4,...,0,24687558,0,2,srodmiescie,tenement,0,0,no,0


In [288]:
numeric_columns = [column for column in combined_df.columns if pd.api.types.is_numeric_dtype(all_platforms_data[column]) and "_nan" not in column]
numeric_columns


['total_area',
 'basement',
 'elevator',
 'balcony',
 'property_level',
 'row_price_m2',
 'total_property_level',
 'building_year',
 'price',
 'is_primary',
 'is_parking',
 'no_rooms']

In [289]:
numeric_columns_outliers = [
 'price',
 'row_price_m2',
 'total_area', 
 'building_year'
]

# Remove outliers

In [290]:
num_rows_start = combined_df.shape[0]
for column in numeric_columns_outliers:
    print(column)
    Q1 = combined_df[column].quantile(0.25)
    Q3 = combined_df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    num_rows_before = combined_df.shape[0]

    outliers = combined_df[(combined_df[column] < lower_bound) | (combined_df[column] > upper_bound)]
    combined_df = combined_df[(combined_df[column] >= lower_bound) & (combined_df[column] <= upper_bound)]
 
    num_rows_after = combined_df.shape[0]
    
    print(f"For column {column}: {num_rows_before} rows before, {num_rows_after} rows after")
    print(f"{num_rows_before - num_rows_after} rows removed")

num_rows_stop = combined_df.shape[0]
final = num_rows_start - num_rows_stop
print(f'DELETED: {final} ROWS, - {round(final /num_rows_start * 100, 2)}%')


price
For column price: 21830 rows before, 20278 rows after
1552 rows removed
row_price_m2
For column row_price_m2: 20278 rows before, 19712 rows after
566 rows removed
total_area
For column total_area: 19712 rows before, 18954 rows after
758 rows removed
building_year
For column building_year: 18954 rows before, 18883 rows after
71 rows removed
DELETED: 2947 ROWS, - 13.5%


In [291]:
combined_df.to_csv('all_platforms_data.csv', index=False)
#underwriting base
all_platforms_data = pd.read_csv('all_platforms_data.csv')
all_platforms_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18883 entries, 0 to 18882
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   is_primary_nan            18883 non-null  int64  
 1   total_area                18883 non-null  float64
 2   basement                  18883 non-null  int64  
 3   elevator                  18883 non-null  int64  
 4   balcony                   18883 non-null  int64  
 5   property_level            18883 non-null  int64  
 6   heating_type_nan          18883 non-null  int64  
 7   balcony_nan               18883 non-null  int64  
 8   row_price_m2              18883 non-null  int64  
 9   total_property_level      18883 non-null  int64  
 10  parking_nan               18883 non-null  int64  
 11  url                       18883 non-null  object 
 12  building_type_nan         18883 non-null  int64  
 13  building_year             18883 non-null  int64  
 14  heatin